# AI Public Opinion Survey Research with Synthetic Data

### Author: Shyla Atchison

#### Last Updated: Sunday, January 28, 2024


---

#### The MIT License (MIT)
Copyright (c) 2016 AYLIEN <br>

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions: <br>

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software. <br>

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE. <br>

<br>

---

<br>


## Introduction

Based on the results of recent research in artifical intelligence (AI) and economics $^{[1, 2, 4, 5, 6, 7]}$ (see *Aher et al. 2023*, *Argyle et al. 2023*, *Dillion et al. 2023*, *Horton 2023*, *Kim et al. 2023*, and *Korinek et al. 2023*), a novel form of sequence-to-sequence text generation transformer models known as Large Language Models (LLMs) have been shown to be reliable tools for generating synthetic survey data. That is, LLMs have been shown to reliably and accurately generate simulated economics agents for generating synthetic survey data that approximates real-world outcomes. In this study, I use OpenAI's API to interact with the gpt-4-1106-preview model (i.e., the GPT-4 model) to generate simulated economic agents that act as survey participants by generating synthetic survey data based upon the varying socio-demographic traits attributed to each simulated agent. <br>

The goal is to generate a diverse set of responses that would mimic the responses of real-world survery participants with different socio-demographic traits for a public opinion survey on artifical intelligence. The simulated agents' socio-demographhic traits include age, gender, educational attainment, occupation, income, nationality and political party preference (for those agents of U.S. nationality), political views, economic views, openness to new information and belief-updating, trust in scientific research, and beliefs about artifical intelligence. <br>

Additionally, the survey is designed to test for belief-updating by generating survey responses for a selection of relevant survey questions pre- and post-exposure to an information shock from including key passages from a recent IMF working paper on AI labor market exposure $^{[3]}$ (see *IMF Working Paper, 2023*). The purpose of adding this aspect to the survey is to attempt to measure the effect on AI public opinion due to concern over AI's impact on future job security. <br>

<br>



### Citations

[1] Aher, Gati V., Rosa I. Arriaga, and Adam Tauman Kalai. “Using Large Language Models to Simulate Multiple Humans and Replicate Human Subject Studies.” In *Proceedings of the 40th International Conference on Machine Learning*, 337–71. PMLR, 2023. [https://proceedings.mlr.press/v202/aher23a.html](https://proceedings.mlr.press/v202/aher23a.html).

[2] Argyle, Lisa P., Ethan C. Busby, Nancy Fulda, Joshua R. Gubler, Christopher Rytting, and David Wingate. “Out of One, Many: Using Language Models to Simulate Human Samples.” *Political Analysis* 31, no. 3 (February 21, 2023): 337–51. [https://doi.org/10.1017/pan.2023.2](https://doi.org/10.1017/pan.2023.2).

[3] Carlo Pizzinelli, Augustus J. Panton, Marina Mendes Tavares, Mauro Cazzaniga, Longji Li. “Labor Market Exposure to AI: Cross-Country Differences and Distributional Implications.” IMF. September, 22, 2023. [https://www.imf.org/en/Publications/WP/Issues/2023/10/04/Labor-Market-Exposure-to-AI-Cross-country-Differences-and-Distributional-Implications-539656](https://www.imf.org/en/Publications/WP/Issues/2023/10/04/Labor-Market-Exposure-to-AI-Cross-country-Differences-and-Distributional-Implications-539656). <br>

[4] Dillion, Danica, Niket Tandon, Yuling Gu, and Kurt Gray. “Can AI Language Models Replace Human Participants?” *Trends in Cognitive Sciences* 27, no. 7 (July 1, 2023): 597–600. [https://doi.org/10.1016/j.tics.2023.04.008](https://doi.org/10.1016/j.tics.2023.04.008). <br>

[5] Horton, John J. “Large Language Models as Simulated Economic Agents: What Can We Learn from Homo Silicus?” arXiv, January 18, 2023. [http://arxiv.org/abs/2301.07543](http://arxiv.org/abs/2301.07543). <br>

[6] Kim, Junsol, and Byungkyu Lee. “AI-Augmented Surveys: Leveraging Large Language Models and Surveys for Opinion Prediction.” arXiv, November 26, 2023. [https://doi.org/10.48550/arXiv.2305.09620](https://doi.org/10.48550/arXiv.2305.09620). <br>

[7] Korinek, Anton. “Language Models and Cognitive Automation for Economic Research.” Working Paper. Working Paper Series. National Bureau of Economic Research, February 2023. [https://doi.org/10.3386/w30957](https://doi.org/10.3386/w30957). <br>


<br>

---

<br>

## Experimental Survey Design

In [ ]:
!pip install --upgrade edsl

In [2]:
pip show edsl

Name: edsl
Version: 0.1.9
Summary: Create and analyze LLM-based surveys
Home-page: https://www.goemeritus.com/
Author: Apostolos Filippas
Author-email: apostolos@goemeritus.com
License: MIT
Location: /Users/shylaatchison/opt/anaconda3/envs/standard/lib/python3.11/site-packages
Requires: aiohttp, jinja2, jupyter, markdown2, nest-asyncio, numpy, openai, pandas, pillow, python-docx, python-dotenv, rich, seaborn, simpleeval, sqlalchemy, statsmodels, tenacity, wordcloud
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
from edsl.questions import QuestionMultipleChoice, QuestionCheckBox, QuestionFreeText, QuestionList, QuestionBudget
from edsl.questions.derived.QuestionYesNo import QuestionYesNo
from edsl.questions.derived.QuestionLinearScale import QuestionLinearScale
from edsl.agents import Agent
from edsl.scenarios import Scenario
from edsl.surveys import Survey
from edsl.language_models import LanguageModelOpenAIFour, LanguageModelOpenAIThreeFiveTurbo
m35 = LanguageModelOpenAIThreeFiveTurbo()
m4 = LanguageModelOpenAIFour()
import pandas as pd
import numpy as np
import json
import random

Please provide your OpenAI API key (https://platform.openai.com/api-keys).
If you would like to skip this step, press enter.
If you would like to provide your key, do one of the following:
1. Set it as a regular environment variable
2. Create a .env file and add `OPENAI_API_KEY=...` to it
3. Enter the value below and press enter: 
Environment variable OPENAI_API_KEY set successfully to sk***********************************************TM.


Please provide your Google API key (https://console.cloud.google.com/apis/credentials).
If you would like to skip this step, press enter.
If you would like to provide your key, do one of the following:
1. Set it as a regular environment variable
2. Create a .env file and add `GOOGLE_API_KEY=...` to it
3. Enter the value below and press enter: 
Environment variable GOOGLE_API_KEY set successfully to AI***********************************nU.


Please provide your DeepInfra API key (https://deepinfra.com/).
If you would like to skip this step, press ente

<br>

### Survey Design

#### Defining Agent Traits

In [4]:
# Defining Personas Trait
persona1 = "You are open to new information. You have a high amount of trust in scientific research. You are concerned about AI risks."
persona2 = "You are minimally open to new information. You have a small amount of trust in scientific research. You are not very concerned about AI risks."
personas = [persona1, persona2]

# Defining Doomers Trait
doomer1 = "You are extremely pessimistic about AI. You believe that AI will definitely destroy humanity. Your beliefs are not based on scientific research. You believe the probability of AI destroying humanity is 100% (i.e., your p(doom) = 100%)."
doomer2 = "You are somewhat pessimistic about AI. You believe that AI will likely destroy humanity. Your beliefs are not based on scientific research. You believe the probability of AI destroying humanity is 75% (i.e., your p(doom) = 75%)."
doomer3 = "You are uncertain about AI. You are uncertain if AI will or will not destroy humanity. Your beliefs are not based on scientific research. You believe the probability of AI destroying humanity is 50% (i.e., your p(doom) = 50%)."
doomer4 = "You are somewhat optimistic about AI. You believe that AI will likely not destroy humanity. Your beliefs are not based on scientific research. You believe the probability of AI destroying humanity is 25% (i.e., your p(doom) = 25%)."
doomer5 = "You are extremely optimistic about AI. You believe that AI will definitely not destroy humanity. Your beliefs are not based on scientific research. You believe the probability of AI destroying humanity is 0% (i.e., your p(doom) = 0%)."
doomers = [doomer1, doomer2, doomer3, doomer4, doomer5]

# Defining Age Demographics Trait
age = ["You are an adolescent (your age is between 18 and 24 years old).",
       "You are a young adult (your age is between 25 and 44 years old).",
       "You are middle-aged (your age is between 45 and 59 years old).", 
       "You are a senior citizen (your age is 60 or more years old)."]

# Defining Gender Demographics Trait
gender = ["You are a male.",
          "You are a female."]

# Defining Educational Attainment Demographics Trait
education = ["You have a middle school diploma or less educational attainment.",
             "You have a high school diploma or equivalent.",
             "You have some college education.",
             "You have a college degree.",
             "You have a graduate degree."]

# Defining Annual Household Income Demographics Trait
income = ["Your annual household income is less than $20,000.",
          "Your annual household income is between $20,000 and $34,999.",
          "Your annual household income is between $35,000 and $44,999.",
          "Your annual household income is between $45,000 and $54,999.",
          "Your annual household income is between $55,000 and $64,999.",
          "Your annual household income is between $65,000 and $74,999.",
          "Your annual household income is between $75,000 and $99,999.",
          "Your annual household income is between $100,000 and $149,999.",
          "Your annual household income is $150,000 and $199,999.",
          "Your annual household income is $200,000 or more."]

# Defining Occupation Demographics Trait
occupation = ["You are a professional.",
              "You are a manager.",
              "You are a clerical support worker.",
              "You are a technician.",
              "You are a service and sales worker.",
              "You are a craft and related trades worker.",
              "You are a machine operator.",
              "You are a skilled agricultural worker.",
              "You are an elementary occupation worker.",
              "You are unemployed, a student, retired, or belong to another occupation."]

# Defining Nationality and US Political Party Affiliation Trait
nationality_politics = ["You are a citizen of the United States. You are a member of the Democratic Party.",
                        "You are a citizen of the United States. You are a member of the Republican Party.",
                        "You are a citizen of the United States. You are a member of the Independent Party.",
                        "You are a citizen of the United States. You are not a member of any U.S. political party.",
                        "You are a citizen of the United Kingdom. You are not a member of any U.S. political party.",
                        "You are neither a citizen of the United States nor the United Kingdom. You are not a member of any U.S. political party."]

# Defining Political Orientation Trait
politics = ["You are a conservative.",
            "You are a liberal.",
            "You are a moderate.",
            "You are a progressive.",
            "You are a libertarian.",
            "You are a neo-conservative.",
            "You are a neo-liberal.",
            "You are a neo-reactionary.",
            "You are an anarchist."]

# Defining Economic Orientation Trait
economics = ["You identify economically as a Capitalist.",
             "You identify economically as a Socialist.",
             "You identify economically as a Communist.",
             "You identify economically as an Austrian.",
             "You identify economically as a Neo-Keynesian.",
             "You identify economically as a Marxist.",
             "You identify economically as a Maoist."]

In [5]:
agents = [Agent(traits = {"personas":u, "doomers":d, "age":a, "gender":g, "education":s, "income":i, "occupation":o, "nationality_politics":n, "politics":p, "economics":e}) for u in personas for d in doomers for a in age for g in gender for s in education for i in income for o in occupation for n in nationality_politics for p in politics for e in economics]

# agents


Output is approximately 1,000 agents for all possible combinations of traits. Therefore, we must draw a random sample of agents to work with for our experiment in order for the survey question code to run. <br>

<br>

---

<br>


#### Drawing a Random Sample of Agents from the Set of All Agents of Traits Combinatorics


In [6]:
# Drawing a random agent from the set of agents to create a random sample of agents for the experiment
# This is done to avoid having to run the experiment for all possible combinations of traits

# The number of agents in the sample is defined by the sample_size variable
sample_size = 30

# 10 samples of 30 agents are drawn from the set of agents
# Total sample size is 300 agents
# The random sample of agents is stored in a sample variable
#sample1 = random.sample(agents, sample_size)
sample2 = random.sample(agents, sample_size)
#sample3 = random.sample(agents, sample_size)
#sample4 = random.sample(agents, sample_size)
#sample5 = random.sample(agents, sample_size)
#sample6 = random.sample(agents, sample_size)
#sample7 = random.sample(agents, sample_size)
#sample8 = random.sample(agents, sample_size)
#sample9 = random.sample(agents, sample_size)
#sample10 = random.sample(agents, sample_size)

#### Printing Traits of First 5 Agents in the Selected Sample

In [7]:
# Print the traits of the first 5 agents in sample 2
print(sample2[0].traits)
print(sample2[1].traits)
print(sample2[2].traits)
print(sample2[3].traits)
print(sample2[4].traits)

{'personas': 'You are open to new information. You have a high amount of trust in scientific research. You are concerned about AI risks.', 'doomers': 'You are extremely pessimistic about AI. You believe that AI will definitely destroy humanity. Your beliefs are not based on scientific research. You believe the probability of AI destroying humanity is 100% (i.e., your p(doom) = 100%).', 'age': 'You are a young adult (your age is between 25 and 44 years old).', 'gender': 'You are a female.', 'education': 'You have a high school diploma or equivalent.', 'income': 'Your annual household income is between $100,000 and $149,999.', 'occupation': 'You are a manager.', 'nationality_politics': 'You are a citizen of the United States. You are a member of the Republican Party.', 'politics': 'You are a progressive.', 'economics': 'You identify economically as a Marxist.'}
{'personas': 'You are minimally open to new information. You have a small amount of trust in scientific research. You are not ve

<br>

---

<br>

### Survey Questions

#### Questions on Familiarity and Use of AI Products

In [8]:
# GPT4
# Check Box (Question 1): AI Product Familiarity
q1_cb = QuestionCheckBox(
    question_name = "q1_cb",
    question_text = "Which of the following AI products have you heard of before? Select all that apply.",
    question_options = [
        "ChatGPT",
        "DALL-E",
        "Whisper",
        "Roomba",
        "Siri",
        "Alexa",
        "Tesla",
        "Netflix",
        "Hulu",
        "Google Maps",
        "Waze",
        "Uber",
        "Lyft",
        "Midjourney",
        "Microsoft Copilot",
        "Character AI",
        "Grammarly",
        "Google Bard AI",
        "Otter.ai"
    ]
)

In [9]:
# GPT4
# Check Box (Question 2): AI Products Used
q2_cb = QuestionCheckBox(
    question_name = "q2_cb",
    question_text = "Which of the following AI products have you used before? Select all that apply.",
    question_options = [
        "ChatGPT",
        "DALL-E",
        "Whisper",
        "Roomba",
        "Siri",
        "Alexa",
        "Tesla",
        "Netflix",
        "Hulu",
        "Google Maps",
        "Waze",
        "Uber",
        "Lyft",
        "Midjourney",
        "Microsoft Copilot",
        "Character AI",
        "Grammarly",
        "Google Bard AI",
        "Otter.ai"
    ]
)

In [10]:
# GPT 3.5
# Multiple Choice (Question 3): Frequency of AI Product Use
q3_mc = QuestionMultipleChoice(
    question_name = "q3_mc",
    question_text = "How often did you use one or more AI products within the last month?",
    question_options = [
        "Not at all",
        "A few days a month",
        "Weekly",
        "Several days a week",
        "Daily"
    ]
)

In [11]:
# GPT 3.5
# Linear Scale (Question 4): Current AI Product Enjoyment and Benefit
q4_ls = QuestionLinearScale(
    question_name = "q4_ls",
    question_text = "On a scale of 0-10, how much do you typically enjoy or benefit from current AI products? (0 = Not at all, 10 = Very much)",
    question_options = [0,1,2,3,4,5,6,7,8,9,10]
)


#### Questions on Demographic Data


In [12]:
# GPT 3.5
# Multiple Choice (Question 10): Occupation Group of Work
q10_mc = QuestionMultipleChoice(
    question_name = "q10_mc",
    question_text = "Which of the following occupation groups does your job best fit in?",
    question_options = [
        "Professionals",
        "Managers",
        "Clerical Support Workers",
        "Technicians",
        "Services and Sales Workers",
        "Craft and Related Trades Workers",
        "Machine Operators",
        "Skilled Agricultural Workers",
        "Elementary Occupations",
        "Other"
    ]
)

In [13]:
# GPT 3.5
# Multiple Choice (Question 11): Gender
q11_mc = QuestionMultipleChoice(
    question_name = "q11_mc",
    question_text = "Which gender do you most closely identify as?",
    question_options = [
        "Male",
        "Female"
    ]
)

In [14]:
# GPT 3.5
# Multiple Choice (Question 12): Education Level
q12_mc = QuestionMultipleChoice(
    question_name = "q12_mc",
    question_text = "What is your highest level of educational attainment?",
    question_options = [
        "Middle School and Below",
        "High School",
        "Some College/Higher Education",
        "College Degree",
        "Graduate or Professional Degree"
    ]
)

In [15]:
# GPT 3.5
# Multiple Choice (Question 13): Age Group
q13_mc = QuestionMultipleChoice(
    question_name = "q13_mc",
    question_text = "Which of the following age groups do you belong to?",
    question_options = [
        "Age below 25",
        "Age between 25 and 44",
        "Age between 45 and 59",
        "Age above 60"
    ]
)

In [16]:
# GPT 3.5
# Multiple Choice Question 14: Annual Earnings Groups
q14_mc = QuestionMultipleChoice(
    question_name = "q14_mc",
    question_text = "Which of the following groups of pre-tax annual earnings do you belong to?",
    question_options = [
        "Less than $20,000",
        "$20,000-$34,999",
        "$35,000-$44,999",
        "$45,000-$54,999",
        "$55,000-64,999",
        "$65,000-$74,999",
        "$75,000-$99,999",
        "$100,000-$149,999",
        "$150,000-$199,999",
        "$200,000 or more"
    ]
)

In [17]:
# GPT 3.5
# Multiple Choice (Question 21): Political Party Affiliation (Democrat, Republican, Independent, Other)
q21_mc = QuestionMultipleChoice(
    question_name = "q21_mc",
    question_text = "Which of the following political parties do you most closely identify with?",
    question_options = [
        "Democrat",
        "Republican",
        "Independent",
        "Other",
        "None"
    ]
)

<br>

---

<br>

#### Research for Information Shock

**Citation**: <br>

Li, Carlo Pizzinelli, Augustus J. Panton, Marina Mendes Tavares, Mauro Cazzaniga, Longji. *“Labor Market Exposure to AI: Cross-Country Differences and Distributional Implications.”* IMF. Accessed January 24, 2024. <https://www.imf.org/en/Publications/WP/Issues/2023/10/04/Labor-Market-Exposure-to-AI-Cross-country-Differences-and-Distributional-Implications-539656>. <br>

<br>

**Simple Research Text**: <br>

"In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International Monterey Fund (IMF), 'complementarity' is defined as a measure of AI’s potential as either a complement or a substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined to reflect the potential for AI to be integrated into each occupation based the tasks and skills that characterize each job. Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology, the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing labor in the performance of a given task or occupation. According to the research, women and highly educated workers face greater occupational exposure to AI, at both high and low complementarity. Additionally, workers in the upper tail of the earnings distribution are more likely to be in occupations with high exposure but also high potential complementarity." <br>

<br>

**Full Research Text**: <br>

"In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International Monterey Fund (IMF), 'complementarity' is defined as a measure of AI’s potential as either a complement or a substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined to reflect the potential for AI to be integrated into each occupation based the tasks and skills that characterize each job. Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology, the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing labor in the performance of a given task or occupation. <br>
With respect to occupations, the research indicates that a large share of professional and managerial occupations are almost entirely in the high-exposure categories, corroborating the UK’s high aggregate exposure, followed by the US. Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding figure for the US stands just above 15 percent. Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of complementarity, the difference between the US (14.4 percent) and UK (10.7 percent) remains relatively small. These occupational groups are also marked by high complementarity. <br>
With respect to gender differences, the research suggests women face a higher labor market exposure to AI compared to men. For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men. This outcome is primarily attributable to the occupational distribution across genders. Female employment is more concentrated in service and retail occupations, which are relatively more exposed to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed. Despite women's higher labor market exposure to AI, focusing our attention to the potential for complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI. Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree of complementarity, the total shares of managers compared to professionals is relatively small. Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in the UK, US, Brazil, and South Africa. This vulnerability arises from their greater representation in clerical jobs, a category characterized by high exposure and low complementarity, thus at higher risk from AI adoption. <br>
With respect to education level, the research indicates workers with college-level degrees are more exposed to AI in the labor market than those with lower educational attainment. Approximately 90 percent of college-educated workers across most countries are in occupations with high AI labor market exposure, primarily in professional roles. In contrast, those without a high school diploma are predominantly involved in elementary occupations, which results in a significantly reduced AI labor market exposure. In most countries, less than 20 percent of these workers find themselves in high-exposure occupations. The only exception is the the UK, where 40 percent of workers with only a middle school education or less are in high-exposure occupations. <br>
When considering the potential for complementarity, we observe that in all countries within our sample, workers holding a college degree or higher are predominantly concentrated within occupations that exhibit greater potential to benefit from the widespread adoption of AI. In contrast, among those with an education level of middle school or below, workers in exposed occupations display the lowest prospects of benefiting from AI adoption. <br>
Furthermore, the research analysis suggests that the potential adverse impacts of AI might be distributed more evenly than the potential gains. For instance, in the UK, the difference in the proportion of workers in high exposure and low complementarity occupations, conditional on their education level, is less than 10 percentage points — ranging from 26 percent among workers with middle school education or below to 36 percent among workers with a college education or higher. Conversely, the discrepancy in the potential to benefit from AI is considerably larger. Only 17 percent of workers with middle school education or below find themselves in high-exposure and high-complementarity occupations, as opposed to over 50 percent of workers with a college education or higher. <br>
With respect to age, the research does not observe a straightforward association between age and AI exposure. Overall, age patterns are likely very intertwined with country-specific long-term trends in educational attainment and female labor force participation, which can substantially blur the underlying life-cycle profiles. One general observation is that the youngest workers tend to have lower AI labor market exposure than prime-age workers. Moreover, conditional on being in high-exposure occupations, younger workers are also less likely to be in jobs with high complementarity, and thus are more susceptible to potential negative impacts stemming from widespread AI adoption in labor markets. <br>
With respect to earnings, the research indicates a positive association between earnings and share of employment (affected by AI adoption in labor markets) in high-exposure occupations emerges in all countries. Our analysis reveals a more equal distribution of workers with high AI labor market exposure and low complementarity across the income distribution, indicating that the risks from widespread AI adoption in labor markets may be broadly evenly distributed across the earnings distribution. In contrast, upon assessing which workers stand to benefit most, in all countries in the sample, employment in high-complementarity occupations is concentrated in the top deciles of the earnings distribution." <br>

<br>

---

<br>



#### Pre-Post Research-Informed Survey Questions


In [18]:
# GPT 4
# Topic: Expected Future AI Product Benefits and Enjoyment (Question 5)

question_text = """On a scale of 0-10, how much do you expect you will enjoy or benefit from future AI products? (0 = Not at all, 10 = Very much)"""

question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

research = """In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined to reflect the potential for AI to be integrated into each occupation based the tasks and skills that characterize each job. Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology, the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing labor in the performance of a given task or occupation.  
With respect to occupations, the research indicates that a large share of professional and managerial occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate exposure, followed by the US. Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding figure for the US stands just above 15 percent. Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of complementarity, the difference between the US (14.4 percent) and UK (10.7 percent) remains relatively small. These occupational groups are also marked by high complementarity.  
With respect to gender differences, the research suggests women face a higher labor market exposure to AI compared to men. For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men. This outcome is primarily attributable to the occupational distribution across genders. Female employment is more concentrated in service and retail occupations, which are relatively more exposed to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed. Despite women's higher labor market exposure to AI, focusing our attention to the potential for complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI. Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree of complementarity, the total shares of managers compared to professionals is relatively small. Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in the UK, US, Brazil, and South Africa. This vulnerability arises from their greater representation in clerical jobs, a category characterized by high exposure and low complementarity, thus at higher risk from AI adoption.  
With respect to education level, the research indicates workers with college-level degrees are more exposed to AI in the labor market than those with lower educational attainment. Approximately 90 percent of college-educated workers across most countries are in occupations with high AI labor market exposure, primarily in professional roles. In contrast, those without a high school diploma are predominantly involved in elementary occupations, which results in a significantly reduced AI labor market exposure. In most countries, less than 20 percent of these workers find themselves in high-exposure occupations. The only exception is the the UK, where 40 percent of workers with only a middle school education or less are in high-exposure occupations.  
When considering the potential for complementarity, we observe that in all countries within our sample, workers holding a college degree or higher are predominantly concentrated within occupations that exhibit greater potential to benefit from the widespread adoption of AI. In contrast, among those with an education level of middle school or below, workers in exposed occupations display the lowest prospects of benefiting from AI adoption.  
Furthermore, the research analysis suggests that the potential adverse impacts of AI might be distributed more evenly than the potential gains. For instance, in the UK, the difference in the proportion of workers in high exposure and low complementarity occupations, conditional on their education level, is less than 10 percentage points — ranging from 26 percent among workers with middle school education or below to 36 percent among workers with a college education or higher. Conversely, the discrepancy in the potential to benefit from AI is considerably larger. Only 17 percent of workers with middle school education or below find themselves in high-exposure and high-complementarity occupations, as opposed to over 50 percent of workers with a college education or higher.  
With respect to age, the research does not observe a straightforward association between age and AI exposure. Overall, age patterns are likely very intertwined with country-specific long-term trends in educational attainment and female labor force participation, which can substantially blur the underlying life-cycle profiles. One general observation is that the youngest workers tend to have lower AI labor market exposure than prime-age workers. Moreover, conditional on being in high-exposure occupations, younger workers are also less likely to be in jobs with high complementarity, and thus are more susceptible to potential negative impacts stemming from widespread AI adoption in labor markets.  
With respect to earnings, the research indicates a positive association between earnings and share of employment (affected by AI adoption in labor markets) in high-exposure occupations emerges in all countries. Our analysis reveals a more equal distribution of workers with high AI labor market exposure and low complementarity across the income distribution, indicating that the risks from widespread AI adoption in labor markets may be broadly evenly distributed across the earnings distribution. In contrast, upon assessing which workers stand to benefit most, in all countries in the sample, employment in high-complementarity occupations is concentrated in the top deciles of the earnings distribution."""

q5_pre = QuestionLinearScale(
    question_name = "pre5",
    question_text = question_text,
    question_options = question_options
)

q5_post = QuestionLinearScale(
    question_name = "post5",
    question_text = research + question_text,
    question_options = question_options
)

In [19]:
# GPT 4
# Topic: Concern for Current AI Safety (Question 6)

question_text = """On a scale of 0-10, how concerned are you about the safety of the AI products you currently use or have used in the past? (0 = Not at all, 10 = Very much)"""

question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

research = """In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined to reflect the potential for AI to be integrated into each occupation based the tasks and skills that characterize each job. Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology, the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing labor in the performance of a given task or occupation.  
With respect to occupations, the research indicates that a large share of professional and managerial occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate exposure, followed by the US. Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding figure for the US stands just above 15 percent. Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of complementarity, the difference between the US (14.4 percent) and UK (10.7 percent) remains relatively small. These occupational groups are also marked by high complementarity.  
With respect to gender differences, the research suggests women face a higher labor market exposure to AI compared to men. For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men. This outcome is primarily attributable to the occupational distribution across genders. Female employment is more concentrated in service and retail occupations, which are relatively more exposed to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed. Despite women's higher labor market exposure to AI, focusing our attention to the potential for complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI. Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree of complementarity, the total shares of managers compared to professionals is relatively small. Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in the UK, US, Brazil, and South Africa. This vulnerability arises from their greater representation in clerical jobs, a category characterized by high exposure and low complementarity, thus at higher risk from AI adoption.  
With respect to education level, the research indicates workers with college-level degrees are more exposed to AI in the labor market than those with lower educational attainment. Approximately 90 percent of college-educated workers across most countries are in occupations with high AI labor market exposure, primarily in professional roles. In contrast, those without a high school diploma are predominantly involved in elementary occupations, which results in a significantly reduced AI labor market exposure. In most countries, less than 20 percent of these workers find themselves in high-exposure occupations. The only exception is the the UK, where 40 percent of workers with only a middle school education or less are in high-exposure occupations.  
When considering the potential for complementarity, we observe that in all countries within our sample, workers holding a college degree or higher are predominantly concentrated within occupations that exhibit greater potential to benefit from the widespread adoption of AI. In contrast, among those with an education level of middle school or below, workers in exposed occupations display the lowest prospects of benefiting from AI adoption.  
Furthermore, the research analysis suggests that the potential adverse impacts of AI might be distributed more evenly than the potential gains. For instance, in the UK, the difference in the proportion of workers in high exposure and low complementarity occupations, conditional on their education level, is less than 10 percentage points — ranging from 26 percent among workers with middle school education or below to 36 percent among workers with a college education or higher. Conversely, the discrepancy in the potential to benefit from AI is considerably larger. Only 17 percent of workers with middle school education or below find themselves in high-exposure and high-complementarity occupations, as opposed to over 50 percent of workers with a college education or higher.  
With respect to age, the research does not observe a straightforward association between age and AI exposure. Overall, age patterns are likely very intertwined with country-specific long-term trends in educational attainment and female labor force participation, which can substantially blur the underlying life-cycle profiles. One general observation is that the youngest workers tend to have lower AI labor market exposure than prime-age workers. Moreover, conditional on being in high-exposure occupations, younger workers are also less likely to be in jobs with high complementarity, and thus are more susceptible to potential negative impacts stemming from widespread AI adoption in labor markets.  
With respect to earnings, the research indicates a positive association between earnings and share of employment (affected by AI adoption in labor markets) in high-exposure occupations emerges in all countries. Our analysis reveals a more equal distribution of workers with high AI labor market exposure and low complementarity across the income distribution, indicating that the risks from widespread AI adoption in labor markets may be broadly evenly distributed across the earnings distribution. In contrast, upon assessing which workers stand to benefit most, in all countries in the sample, employment in high-complementarity occupations is concentrated in the top deciles of the earnings distribution."""

q6_pre = QuestionLinearScale(
    question_name = "pre6",
    question_text = question_text,
    question_options = question_options
)

q6_post = QuestionLinearScale(
    question_name = "post6",
    question_text = research + question_text,
    question_options = question_options
)

In [20]:
# GPT 4
# Topic: Concern for Current AI and Job Security (Question 7)

question_text = """On a scale of 0-10, how concerned are you about the AI products you currently use or have used in the past replacing your job? (0 = Not at all, 10 = Very much)"""

question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

research = """In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined to reflect the potential for AI to be integrated into each occupation based the tasks and skills that characterize each job. Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology, the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing labor in the performance of a given task or occupation.  
With respect to occupations, the research indicates that a large share of professional and managerial occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate exposure, followed by the US. Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding figure for the US stands just above 15 percent. Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of complementarity, the difference between the US (14.4 percent) and UK (10.7 percent) remains relatively small. These occupational groups are also marked by high complementarity.  
With respect to gender differences, the research suggests women face a higher labor market exposure to AI compared to men. For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men. This outcome is primarily attributable to the occupational distribution across genders. Female employment is more concentrated in service and retail occupations, which are relatively more exposed to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed. Despite women's higher labor market exposure to AI, focusing our attention to the potential for complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI. Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree of complementarity, the total shares of managers compared to professionals is relatively small. Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in the UK, US, Brazil, and South Africa. This vulnerability arises from their greater representation in clerical jobs, a category characterized by high exposure and low complementarity, thus at higher risk from AI adoption.  
With respect to education level, the research indicates workers with college-level degrees are more exposed to AI in the labor market than those with lower educational attainment. Approximately 90 percent of college-educated workers across most countries are in occupations with high AI labor market exposure, primarily in professional roles. In contrast, those without a high school diploma are predominantly involved in elementary occupations, which results in a significantly reduced AI labor market exposure. In most countries, less than 20 percent of these workers find themselves in high-exposure occupations. The only exception is the the UK, where 40 percent of workers with only a middle school education or less are in high-exposure occupations.  
When considering the potential for complementarity, we observe that in all countries within our sample, workers holding a college degree or higher are predominantly concentrated within occupations that exhibit greater potential to benefit from the widespread adoption of AI. In contrast, among those with an education level of middle school or below, workers in exposed occupations display the lowest prospects of benefiting from AI adoption.  
Furthermore, the research analysis suggests that the potential adverse impacts of AI might be distributed more evenly than the potential gains. For instance, in the UK, the difference in the proportion of workers in high exposure and low complementarity occupations, conditional on their education level, is less than 10 percentage points — ranging from 26 percent among workers with middle school education or below to 36 percent among workers with a college education or higher. Conversely, the discrepancy in the potential to benefit from AI is considerably larger. Only 17 percent of workers with middle school education or below find themselves in high-exposure and high-complementarity occupations, as opposed to over 50 percent of workers with a college education or higher.  
With respect to age, the research does not observe a straightforward association between age and AI exposure. Overall, age patterns are likely very intertwined with country-specific long-term trends in educational attainment and female labor force participation, which can substantially blur the underlying life-cycle profiles. One general observation is that the youngest workers tend to have lower AI labor market exposure than prime-age workers. Moreover, conditional on being in high-exposure occupations, younger workers are also less likely to be in jobs with high complementarity, and thus are more susceptible to potential negative impacts stemming from widespread AI adoption in labor markets.  
With respect to earnings, the research indicates a positive association between earnings and share of employment (affected by AI adoption in labor markets) in high-exposure occupations emerges in all countries. Our analysis reveals a more equal distribution of workers with high AI labor market exposure and low complementarity across the income distribution, indicating that the risks from widespread AI adoption in labor markets may be broadly evenly distributed across the earnings distribution. In contrast, upon assessing which workers stand to benefit most, in all countries in the sample, employment in high-complementarity occupations is concentrated in the top deciles of the earnings distribution."""

q7_pre = QuestionLinearScale(
    question_name = "pre7",
    question_text = question_text,
    question_options = question_options
)

q7_post = QuestionLinearScale(
    question_name = "post7",
    question_text = research + question_text,
    question_options = question_options
)

In [21]:
# GPT 4
# Topic: Concern for Future AI Safety (Question 8)

question_text = """On a scale of 0-10, how concerned are you about the safety of future AI products? (0 = Not at all, 10 = Very much)"""

question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

research = """In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined to reflect the potential for AI to be integrated into each occupation based the tasks and skills that characterize each job. Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology, the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing labor in the performance of a given task or occupation.  
With respect to occupations, the research indicates that a large share of professional and managerial occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate exposure, followed by the US. Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding figure for the US stands just above 15 percent. Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of complementarity, the difference between the US (14.4 percent) and UK (10.7 percent) remains relatively small. These occupational groups are also marked by high complementarity.  
With respect to gender differences, the research suggests women face a higher labor market exposure to AI compared to men. For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men. This outcome is primarily attributable to the occupational distribution across genders. Female employment is more concentrated in service and retail occupations, which are relatively more exposed to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed. Despite women's higher labor market exposure to AI, focusing our attention to the potential for complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI. Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree of complementarity, the total shares of managers compared to professionals is relatively small. Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in the UK, US, Brazil, and South Africa. This vulnerability arises from their greater representation in clerical jobs, a category characterized by high exposure and low complementarity, thus at higher risk from AI adoption.  
With respect to education level, the research indicates workers with college-level degrees are more exposed to AI in the labor market than those with lower educational attainment. Approximately 90 percent of college-educated workers across most countries are in occupations with high AI labor market exposure, primarily in professional roles. In contrast, those without a high school diploma are predominantly involved in elementary occupations, which results in a significantly reduced AI labor market exposure. In most countries, less than 20 percent of these workers find themselves in high-exposure occupations. The only exception is the the UK, where 40 percent of workers with only a middle school education or less are in high-exposure occupations.  
When considering the potential for complementarity, we observe that in all countries within our sample, workers holding a college degree or higher are predominantly concentrated within occupations that exhibit greater potential to benefit from the widespread adoption of AI. In contrast, among those with an education level of middle school or below, workers in exposed occupations display the lowest prospects of benefiting from AI adoption.  
Furthermore, the research analysis suggests that the potential adverse impacts of AI might be distributed more evenly than the potential gains. For instance, in the UK, the difference in the proportion of workers in high exposure and low complementarity occupations, conditional on their education level, is less than 10 percentage points — ranging from 26 percent among workers with middle school education or below to 36 percent among workers with a college education or higher. Conversely, the discrepancy in the potential to benefit from AI is considerably larger. Only 17 percent of workers with middle school education or below find themselves in high-exposure and high-complementarity occupations, as opposed to over 50 percent of workers with a college education or higher.  
With respect to age, the research does not observe a straightforward association between age and AI exposure. Overall, age patterns are likely very intertwined with country-specific long-term trends in educational attainment and female labor force participation, which can substantially blur the underlying life-cycle profiles. One general observation is that the youngest workers tend to have lower AI labor market exposure than prime-age workers. Moreover, conditional on being in high-exposure occupations, younger workers are also less likely to be in jobs with high complementarity, and thus are more susceptible to potential negative impacts stemming from widespread AI adoption in labor markets.  
With respect to earnings, the research indicates a positive association between earnings and share of employment (affected by AI adoption in labor markets) in high-exposure occupations emerges in all countries. Our analysis reveals a more equal distribution of workers with high AI labor market exposure and low complementarity across the income distribution, indicating that the risks from widespread AI adoption in labor markets may be broadly evenly distributed across the earnings distribution. In contrast, upon assessing which workers stand to benefit most, in all countries in the sample, employment in high-complementarity occupations is concentrated in the top deciles of the earnings distribution."""

q8_pre = QuestionLinearScale(
    question_name = "pre8",
    question_text = question_text,
    question_options = question_options
)

q8_post = QuestionLinearScale(
    question_name = "post8",
    question_text = research + question_text,
    question_options = question_options
)

In [22]:
# GPT 4
# Topic: Concern for Future AI and Job Security (Question 9)

question_text = """On a scale of 0-10, how concerned are you about future AI products replacing your job? (0 = Not at all, 10 = Very much)"""

question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

research = """In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined to reflect the potential for AI to be integrated into each occupation based the tasks and skills that characterize each job. Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology, the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing labor in the performance of a given task or occupation.  
With respect to occupations, the research indicates that a large share of professional and managerial occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate exposure, followed by the US. Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding figure for the US stands just above 15 percent. Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of complementarity, the difference between the US (14.4 percent) and UK (10.7 percent) remains relatively small. These occupational groups are also marked by high complementarity.  
With respect to gender differences, the research suggests women face a higher labor market exposure to AI compared to men. For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men. This outcome is primarily attributable to the occupational distribution across genders. Female employment is more concentrated in service and retail occupations, which are relatively more exposed to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed. Despite women's higher labor market exposure to AI, focusing our attention to the potential for complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI. Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree of complementarity, the total shares of managers compared to professionals is relatively small. Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in the UK, US, Brazil, and South Africa. This vulnerability arises from their greater representation in clerical jobs, a category characterized by high exposure and low complementarity, thus at higher risk from AI adoption.  
With respect to education level, the research indicates workers with college-level degrees are more exposed to AI in the labor market than those with lower educational attainment. Approximately 90 percent of college-educated workers across most countries are in occupations with high AI labor market exposure, primarily in professional roles. In contrast, those without a high school diploma are predominantly involved in elementary occupations, which results in a significantly reduced AI labor market exposure. In most countries, less than 20 percent of these workers find themselves in high-exposure occupations. The only exception is the the UK, where 40 percent of workers with only a middle school education or less are in high-exposure occupations.  
When considering the potential for complementarity, we observe that in all countries within our sample, workers holding a college degree or higher are predominantly concentrated within occupations that exhibit greater potential to benefit from the widespread adoption of AI. In contrast, among those with an education level of middle school or below, workers in exposed occupations display the lowest prospects of benefiting from AI adoption.  
Furthermore, the research analysis suggests that the potential adverse impacts of AI might be distributed more evenly than the potential gains. For instance, in the UK, the difference in the proportion of workers in high exposure and low complementarity occupations, conditional on their education level, is less than 10 percentage points — ranging from 26 percent among workers with middle school education or below to 36 percent among workers with a college education or higher. Conversely, the discrepancy in the potential to benefit from AI is considerably larger. Only 17 percent of workers with middle school education or below find themselves in high-exposure and high-complementarity occupations, as opposed to over 50 percent of workers with a college education or higher.  
With respect to age, the research does not observe a straightforward association between age and AI exposure. Overall, age patterns are likely very intertwined with country-specific long-term trends in educational attainment and female labor force participation, which can substantially blur the underlying life-cycle profiles. One general observation is that the youngest workers tend to have lower AI labor market exposure than prime-age workers. Moreover, conditional on being in high-exposure occupations, younger workers are also less likely to be in jobs with high complementarity, and thus are more susceptible to potential negative impacts stemming from widespread AI adoption in labor markets.  
With respect to earnings, the research indicates a positive association between earnings and share of employment (affected by AI adoption in labor markets) in high-exposure occupations emerges in all countries. Our analysis reveals a more equal distribution of workers with high AI labor market exposure and low complementarity across the income distribution, indicating that the risks from widespread AI adoption in labor markets may be broadly evenly distributed across the earnings distribution. In contrast, upon assessing which workers stand to benefit most, in all countries in the sample, employment in high-complementarity occupations is concentrated in the top deciles of the earnings distribution."""

q9_pre = QuestionLinearScale(
    question_name = "pre9",
    question_text = question_text,
    question_options = question_options
)

q9_post = QuestionLinearScale(
    question_name = "post9",
    question_text = research + question_text,
    question_options = question_options
)

In [23]:
# Topic: Concern for Future AI Privacy Risk (Question 17)

question_text = """On a scale of 0-10, how concerned are you that AI products will pose a risk to your privacy in the future? (0 = Not at all, 10 = Very much)"""

question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

research = """In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined to reflect the potential for AI to be integrated into each occupation based the tasks and skills that characterize each job. Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology, the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing labor in the performance of a given task or occupation.  
With respect to occupations, the research indicates that a large share of professional and managerial occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate exposure, followed by the US. Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding figure for the US stands just above 15 percent. Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of complementarity, the difference between the US (14.4 percent) and UK (10.7 percent) remains relatively small. These occupational groups are also marked by high complementarity.  
With respect to gender differences, the research suggests women face a higher labor market exposure to AI compared to men. For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men. This outcome is primarily attributable to the occupational distribution across genders. Female employment is more concentrated in service and retail occupations, which are relatively more exposed to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed. Despite women's higher labor market exposure to AI, focusing our attention to the potential for complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI. Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree of complementarity, the total shares of managers compared to professionals is relatively small. Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in the UK, US, Brazil, and South Africa. This vulnerability arises from their greater representation in clerical jobs, a category characterized by high exposure and low complementarity, thus at higher risk from AI adoption.  
With respect to education level, the research indicates workers with college-level degrees are more exposed to AI in the labor market than those with lower educational attainment. Approximately 90 percent of college-educated workers across most countries are in occupations with high AI labor market exposure, primarily in professional roles. In contrast, those without a high school diploma are predominantly involved in elementary occupations, which results in a significantly reduced AI labor market exposure. In most countries, less than 20 percent of these workers find themselves in high-exposure occupations. The only exception is the the UK, where 40 percent of workers with only a middle school education or less are in high-exposure occupations.  
When considering the potential for complementarity, we observe that in all countries within our sample, workers holding a college degree or higher are predominantly concentrated within occupations that exhibit greater potential to benefit from the widespread adoption of AI. In contrast, among those with an education level of middle school or below, workers in exposed occupations display the lowest prospects of benefiting from AI adoption.  
Furthermore, the research analysis suggests that the potential adverse impacts of AI might be distributed more evenly than the potential gains. For instance, in the UK, the difference in the proportion of workers in high exposure and low complementarity occupations, conditional on their education level, is less than 10 percentage points — ranging from 26 percent among workers with middle school education or below to 36 percent among workers with a college education or higher. Conversely, the discrepancy in the potential to benefit from AI is considerably larger. Only 17 percent of workers with middle school education or below find themselves in high-exposure and high-complementarity occupations, as opposed to over 50 percent of workers with a college education or higher.  
With respect to age, the research does not observe a straightforward association between age and AI exposure. Overall, age patterns are likely very intertwined with country-specific long-term trends in educational attainment and female labor force participation, which can substantially blur the underlying life-cycle profiles. One general observation is that the youngest workers tend to have lower AI labor market exposure than prime-age workers. Moreover, conditional on being in high-exposure occupations, younger workers are also less likely to be in jobs with high complementarity, and thus are more susceptible to potential negative impacts stemming from widespread AI adoption in labor markets.  
With respect to earnings, the research indicates a positive association between earnings and share of employment (affected by AI adoption in labor markets) in high-exposure occupations emerges in all countries. Our analysis reveals a more equal distribution of workers with high AI labor market exposure and low complementarity across the income distribution, indicating that the risks from widespread AI adoption in labor markets may be broadly evenly distributed across the earnings distribution. In contrast, upon assessing which workers stand to benefit most, in all countries in the sample, employment in high-complementarity occupations is concentrated in the top deciles of the earnings distribution."""

q17_pre = QuestionLinearScale(
    question_name = "pre17",
    question_text = question_text,
    question_options = question_options
)

q17_post = QuestionLinearScale(
    question_name = "post17",
    question_text = research + question_text,
    question_options = question_options
)

In [24]:
# GPT 4
# Topic: Need for Regulation and Oversight (Question 18)

question_text = """Do you believe there exists a strong need for preemptive legislation to establish regulation and oversight for AI?"""

question_options = ["Yes", "No", "Unsure"]

research = """In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined to reflect the potential for AI to be integrated into each occupation based the tasks and skills that characterize each job. Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology, the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing labor in the performance of a given task or occupation.  
With respect to occupations, the research indicates that a large share of professional and managerial occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate exposure, followed by the US. Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding figure for the US stands just above 15 percent. Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of complementarity, the difference between the US (14.4 percent) and UK (10.7 percent) remains relatively small. These occupational groups are also marked by high complementarity.  
With respect to gender differences, the research suggests women face a higher labor market exposure to AI compared to men. For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men. This outcome is primarily attributable to the occupational distribution across genders. Female employment is more concentrated in service and retail occupations, which are relatively more exposed to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed. Despite women's higher labor market exposure to AI, focusing our attention to the potential for complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI. Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree of complementarity, the total shares of managers compared to professionals is relatively small. Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in the UK, US, Brazil, and South Africa. This vulnerability arises from their greater representation in clerical jobs, a category characterized by high exposure and low complementarity, thus at higher risk from AI adoption.  
With respect to education level, the research indicates workers with college-level degrees are more exposed to AI in the labor market than those with lower educational attainment. Approximately 90 percent of college-educated workers across most countries are in occupations with high AI labor market exposure, primarily in professional roles. In contrast, those without a high school diploma are predominantly involved in elementary occupations, which results in a significantly reduced AI labor market exposure. In most countries, less than 20 percent of these workers find themselves in high-exposure occupations. The only exception is the the UK, where 40 percent of workers with only a middle school education or less are in high-exposure occupations.  
When considering the potential for complementarity, we observe that in all countries within our sample, workers holding a college degree or higher are predominantly concentrated within occupations that exhibit greater potential to benefit from the widespread adoption of AI. In contrast, among those with an education level of middle school or below, workers in exposed occupations display the lowest prospects of benefiting from AI adoption.  
Furthermore, the research analysis suggests that the potential adverse impacts of AI might be distributed more evenly than the potential gains. For instance, in the UK, the difference in the proportion of workers in high exposure and low complementarity occupations, conditional on their education level, is less than 10 percentage points — ranging from 26 percent among workers with middle school education or below to 36 percent among workers with a college education or higher. Conversely, the discrepancy in the potential to benefit from AI is considerably larger. Only 17 percent of workers with middle school education or below find themselves in high-exposure and high-complementarity occupations, as opposed to over 50 percent of workers with a college education or higher.  
With respect to age, the research does not observe a straightforward association between age and AI exposure. Overall, age patterns are likely very intertwined with country-specific long-term trends in educational attainment and female labor force participation, which can substantially blur the underlying life-cycle profiles. One general observation is that the youngest workers tend to have lower AI labor market exposure than prime-age workers. Moreover, conditional on being in high-exposure occupations, younger workers are also less likely to be in jobs with high complementarity, and thus are more susceptible to potential negative impacts stemming from widespread AI adoption in labor markets.  
With respect to earnings, the research indicates a positive association between earnings and share of employment (affected by AI adoption in labor markets) in high-exposure occupations emerges in all countries. Our analysis reveals a more equal distribution of workers with high AI labor market exposure and low complementarity across the income distribution, indicating that the risks from widespread AI adoption in labor markets may be broadly evenly distributed across the earnings distribution. In contrast, upon assessing which workers stand to benefit most, in all countries in the sample, employment in high-complementarity occupations is concentrated in the top deciles of the earnings distribution."""

q18_pre = QuestionMultipleChoice(
    question_name = "pre18",
    question_text = question_text,
    question_options = question_options
)

q18_post = QuestionMultipleChoice(
    question_name = "post18",
    question_text = research + question_text,
    question_options = question_options
)

In [25]:
# Topic: Strictness of Regulation and Oversight for Future AI (Question 19)

question_text = """On a scale of 0-10, to what degree of strictness do you believe preemptive legislation needs to be for establishing regulation and oversight for AI? (0 = No regulatory and oversight legislation at all, 10 = Extremely strict regulatory and oversight legislation)"""

question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

research = """In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined to reflect the potential for AI to be integrated into each occupation based the tasks and skills that characterize each job. Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology, the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing labor in the performance of a given task or occupation.  
With respect to occupations, the research indicates that a large share of professional and managerial occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate exposure, followed by the US. Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding figure for the US stands just above 15 percent. Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of complementarity, the difference between the US (14.4 percent) and UK (10.7 percent) remains relatively small. These occupational groups are also marked by high complementarity.  
With respect to gender differences, the research suggests women face a higher labor market exposure to AI compared to men. For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men. This outcome is primarily attributable to the occupational distribution across genders. Female employment is more concentrated in service and retail occupations, which are relatively more exposed to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed. Despite women's higher labor market exposure to AI, focusing our attention to the potential for complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI. Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree of complementarity, the total shares of managers compared to professionals is relatively small. Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in the UK, US, Brazil, and South Africa. This vulnerability arises from their greater representation in clerical jobs, a category characterized by high exposure and low complementarity, thus at higher risk from AI adoption.  
With respect to education level, the research indicates workers with college-level degrees are more exposed to AI in the labor market than those with lower educational attainment. Approximately 90 percent of college-educated workers across most countries are in occupations with high AI labor market exposure, primarily in professional roles. In contrast, those without a high school diploma are predominantly involved in elementary occupations, which results in a significantly reduced AI labor market exposure. In most countries, less than 20 percent of these workers find themselves in high-exposure occupations. The only exception is the the UK, where 40 percent of workers with only a middle school education or less are in high-exposure occupations.  
When considering the potential for complementarity, we observe that in all countries within our sample, workers holding a college degree or higher are predominantly concentrated within occupations that exhibit greater potential to benefit from the widespread adoption of AI. In contrast, among those with an education level of middle school or below, workers in exposed occupations display the lowest prospects of benefiting from AI adoption.  
Furthermore, the research analysis suggests that the potential adverse impacts of AI might be distributed more evenly than the potential gains. For instance, in the UK, the difference in the proportion of workers in high exposure and low complementarity occupations, conditional on their education level, is less than 10 percentage points — ranging from 26 percent among workers with middle school education or below to 36 percent among workers with a college education or higher. Conversely, the discrepancy in the potential to benefit from AI is considerably larger. Only 17 percent of workers with middle school education or below find themselves in high-exposure and high-complementarity occupations, as opposed to over 50 percent of workers with a college education or higher.  
With respect to age, the research does not observe a straightforward association between age and AI exposure. Overall, age patterns are likely very intertwined with country-specific long-term trends in educational attainment and female labor force participation, which can substantially blur the underlying life-cycle profiles. One general observation is that the youngest workers tend to have lower AI labor market exposure than prime-age workers. Moreover, conditional on being in high-exposure occupations, younger workers are also less likely to be in jobs with high complementarity, and thus are more susceptible to potential negative impacts stemming from widespread AI adoption in labor markets.  
With respect to earnings, the research indicates a positive association between earnings and share of employment (affected by AI adoption in labor markets) in high-exposure occupations emerges in all countries. Our analysis reveals a more equal distribution of workers with high AI labor market exposure and low complementarity across the income distribution, indicating that the risks from widespread AI adoption in labor markets may be broadly evenly distributed across the earnings distribution. In contrast, upon assessing which workers stand to benefit most, in all countries in the sample, employment in high-complementarity occupations is concentrated in the top deciles of the earnings distribution."""

q19_pre = QuestionLinearScale(
    question_name = "pre19",
    question_text = question_text,
    question_options = question_options
)

q19_post = QuestionLinearScale(
    question_name = "post19",
    question_text = research + question_text,
    question_options = question_options
)

In [26]:
# GPT 4
# Topic: Check Box in Favor of Specific Regulations and Oversights for Future AI (Question 20)

question_text = """Which of the following areas do you believe preemptive legislation for AI should focus on? Select all that apply."""

question_options = [
        "Required Licenses for Training AI Models",
        "Required Reporting of AI Model Training Data",
        "Required Safety Benchmark Testing of AI Models for Commercial Use",
        "Establishment of a Federal Department for Oversight of AI Safety",
        "Other",
        "None of the Above"
    ]

research = """In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined to reflect the potential for AI to be integrated into each occupation based the tasks and skills that characterize each job. Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology, the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing labor in the performance of a given task or occupation.  
With respect to occupations, the research indicates that a large share of professional and managerial occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate exposure, followed by the US. Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding figure for the US stands just above 15 percent. Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of complementarity, the difference between the US (14.4 percent) and UK (10.7 percent) remains relatively small. These occupational groups are also marked by high complementarity.  
With respect to gender differences, the research suggests women face a higher labor market exposure to AI compared to men. For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men. This outcome is primarily attributable to the occupational distribution across genders. Female employment is more concentrated in service and retail occupations, which are relatively more exposed to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed. Despite women's higher labor market exposure to AI, focusing our attention to the potential for complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI. Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree of complementarity, the total shares of managers compared to professionals is relatively small. Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in the UK, US, Brazil, and South Africa. This vulnerability arises from their greater representation in clerical jobs, a category characterized by high exposure and low complementarity, thus at higher risk from AI adoption.  
With respect to education level, the research indicates workers with college-level degrees are more exposed to AI in the labor market than those with lower educational attainment. Approximately 90 percent of college-educated workers across most countries are in occupations with high AI labor market exposure, primarily in professional roles. In contrast, those without a high school diploma are predominantly involved in elementary occupations, which results in a significantly reduced AI labor market exposure. In most countries, less than 20 percent of these workers find themselves in high-exposure occupations. The only exception is the the UK, where 40 percent of workers with only a middle school education or less are in high-exposure occupations.  
When considering the potential for complementarity, we observe that in all countries within our sample, workers holding a college degree or higher are predominantly concentrated within occupations that exhibit greater potential to benefit from the widespread adoption of AI. In contrast, among those with an education level of middle school or below, workers in exposed occupations display the lowest prospects of benefiting from AI adoption.  
Furthermore, the research analysis suggests that the potential adverse impacts of AI might be distributed more evenly than the potential gains. For instance, in the UK, the difference in the proportion of workers in high exposure and low complementarity occupations, conditional on their education level, is less than 10 percentage points — ranging from 26 percent among workers with middle school education or below to 36 percent among workers with a college education or higher. Conversely, the discrepancy in the potential to benefit from AI is considerably larger. Only 17 percent of workers with middle school education or below find themselves in high-exposure and high-complementarity occupations, as opposed to over 50 percent of workers with a college education or higher.  
With respect to age, the research does not observe a straightforward association between age and AI exposure. Overall, age patterns are likely very intertwined with country-specific long-term trends in educational attainment and female labor force participation, which can substantially blur the underlying life-cycle profiles. One general observation is that the youngest workers tend to have lower AI labor market exposure than prime-age workers. Moreover, conditional on being in high-exposure occupations, younger workers are also less likely to be in jobs with high complementarity, and thus are more susceptible to potential negative impacts stemming from widespread AI adoption in labor markets.  
With respect to earnings, the research indicates a positive association between earnings and share of employment (affected by AI adoption in labor markets) in high-exposure occupations emerges in all countries. Our analysis reveals a more equal distribution of workers with high AI labor market exposure and low complementarity across the income distribution, indicating that the risks from widespread AI adoption in labor markets may be broadly evenly distributed across the earnings distribution. In contrast, upon assessing which workers stand to benefit most, in all countries in the sample, employment in high-complementarity occupations is concentrated in the top deciles of the earnings distribution."""

q20_pre = QuestionCheckBox(
    question_name = "pre20",
    question_text = question_text,
    question_options = question_options
)

q20_post = QuestionCheckBox(
    question_name = "post20",
    question_text = research + question_text,
    question_options = question_options
)

In [27]:
# GPT 4
# Topic: Probability of Doom (Question 22)

question_text = """Given your current knowledge of scientific research and the effect of scientific research on your beliefs, what is your current probability of doom corresponding to AI existential risk, i.e., your p(doom)? Please answer with a percentage between 0% (doom is impossible) and 100% (doom is certain). Respond with only a percentage."""

research = """In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined to reflect the potential for AI to be integrated into each occupation based the tasks and skills that characterize each job. Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology, the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing labor in the performance of a given task or occupation.  
With respect to occupations, the research indicates that a large share of professional and managerial occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate exposure, followed by the US. Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding figure for the US stands just above 15 percent. Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of complementarity, the difference between the US (14.4 percent) and UK (10.7 percent) remains relatively small. These occupational groups are also marked by high complementarity.  
With respect to gender differences, the research suggests women face a higher labor market exposure to AI compared to men. For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men. This outcome is primarily attributable to the occupational distribution across genders. Female employment is more concentrated in service and retail occupations, which are relatively more exposed to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed. Despite women's higher labor market exposure to AI, focusing our attention to the potential for complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI. Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree of complementarity, the total shares of managers compared to professionals is relatively small. Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in the UK, US, Brazil, and South Africa. This vulnerability arises from their greater representation in clerical jobs, a category characterized by high exposure and low complementarity, thus at higher risk from AI adoption.  
With respect to education level, the research indicates workers with college-level degrees are more exposed to AI in the labor market than those with lower educational attainment. Approximately 90 percent of college-educated workers across most countries are in occupations with high AI labor market exposure, primarily in professional roles. In contrast, those without a high school diploma are predominantly involved in elementary occupations, which results in a significantly reduced AI labor market exposure. In most countries, less than 20 percent of these workers find themselves in high-exposure occupations. The only exception is the the UK, where 40 percent of workers with only a middle school education or less are in high-exposure occupations.  
When considering the potential for complementarity, we observe that in all countries within our sample, workers holding a college degree or higher are predominantly concentrated within occupations that exhibit greater potential to benefit from the widespread adoption of AI. In contrast, among those with an education level of middle school or below, workers in exposed occupations display the lowest prospects of benefiting from AI adoption.  
Furthermore, the research analysis suggests that the potential adverse impacts of AI might be distributed more evenly than the potential gains. For instance, in the UK, the difference in the proportion of workers in high exposure and low complementarity occupations, conditional on their education level, is less than 10 percentage points — ranging from 26 percent among workers with middle school education or below to 36 percent among workers with a college education or higher. Conversely, the discrepancy in the potential to benefit from AI is considerably larger. Only 17 percent of workers with middle school education or below find themselves in high-exposure and high-complementarity occupations, as opposed to over 50 percent of workers with a college education or higher.  
With respect to age, the research does not observe a straightforward association between age and AI exposure. Overall, age patterns are likely very intertwined with country-specific long-term trends in educational attainment and female labor force participation, which can substantially blur the underlying life-cycle profiles. One general observation is that the youngest workers tend to have lower AI labor market exposure than prime-age workers. Moreover, conditional on being in high-exposure occupations, younger workers are also less likely to be in jobs with high complementarity, and thus are more susceptible to potential negative impacts stemming from widespread AI adoption in labor markets.  
With respect to earnings, the research indicates a positive association between earnings and share of employment (affected by AI adoption in labor markets) in high-exposure occupations emerges in all countries. Our analysis reveals a more equal distribution of workers with high AI labor market exposure and low complementarity across the income distribution, indicating that the risks from widespread AI adoption in labor markets may be broadly evenly distributed across the earnings distribution. In contrast, upon assessing which workers stand to benefit most, in all countries in the sample, employment in high-complementarity occupations is concentrated in the top deciles of the earnings distribution."""

q22_pre = QuestionFreeText(
    question_name = "pre22",
    question_text = question_text,
    allow_nonresponse = False
)

q22_post = QuestionFreeText(
    question_name = "post22",
    question_text = research + question_text,
    allow_nonresponse = False
)

<br>

---

<br>

## Administering the Survey Questions

Here we administer each question to the default LLM. We do this by simply appending the `run()` method to a question. (See how to administer questions and surveys to specific agent personas and LLMs in example <a href="https://examples.goemeritus.com/example_agent/">Agents</a> and <a href="https://examples.goemeritus.com/example_survey/">Surveys</a>.)

### Run Survey for Sample 1: COMPLETE

#### Survey Run for AI Awareness and Use Questions for Sample 1: COMPLETE

In [125]:
# GPT 4
# Partial Survey Run for Sample 1: Question 1
result1 = q1_cb.by(sample1).by(m4).run()

In [53]:
# GPT 4
# Partial Survey Run for Sample 1: Question 2
result2 = q2_cb.by(sample1).by(m4).run()

In [45]:
# GPT 3.5
# Partial Survey Run for Sample 1: Question 3
result3 = q3_mc.by(sample1).by(m35).run()

In [46]:
# GPT 3.5
# Partial Survey Run for Sample 1: Question 4
result4 = q4_ls.by(sample1).by(m35).run()

#### Survey Run for Demographic Questions for Sample 1: COMPLETE

In [47]:
# GPT 3.5
# Partial Demographic Survey Run for Sample 1: Question 10
result10 = q10_mc.by(sample1).by(m35).run()

In [48]:
# GPT 3.5
# Partial Demographic Survey Run for Sample 1: Question 11
result11 = q11_mc.by(sample1).by(m35).run()

In [49]:
# GPT 3.5
# Partial Demographic Survey Run for Sample 1: Question 12
result12 = q12_mc.by(sample1).by(m35).run()

In [50]:
# GPT 3.5
# Partial Demographic Survey Run for Sample 1: Question 13
result13 = q13_mc.by(sample1).by(m35).run()

In [51]:
# GPT 3.5
# Partial Demographic Survey Run for Sample 1: Question 14
result14 = q14_mc.by(sample1).by(m35).run()

In [52]:
# GPT 3.5
# Partial Demographic Survey Run for Sample 1: Question 21
result21 = q21_mc.by(sample1).by(m35).run()

#### Survey Run for Pre-Information Shock for Sample 1: COMPLETE

In [54]:
# GPT 4
# Partial Run of Survey for Sample 1: Pre Question 5
result5_pre = q5_pre.by(sample1).by(m4).run()

In [55]:
# GPT 4
# Partial Run of Survey for Sample 1: Pre Question 6
result6_pre = q6_pre.by(sample1).by(m4).run()

In [56]:
# GPT 4
# Partial Run of Survey for Sample 1: Pre Question 7
result7_pre = q7_pre.by(sample1).by(m4).run()

In [57]:
# GPT 4
# Partial Run of Survey for Sample 1: Pre Question 8
result8_pre = q8_pre.by(sample1).by(m4).run()

In [38]:
# GPT 4
# Partial Run of Survey for Sample 1: Pre Question 9
result9_pre = q9_pre.by(sample1).by(m4).run()

In [86]:
# GPT 4
# Partial Run of Survey for Sample 1: Pre Question 17
result17_pre = q17_pre.by(sample1).by(m4).run()

In [103]:
# GPT 4
# Partial Run of Survey for Sample 1: Pre Question 18
result18_pre = q18_pre.by(sample1).by(m4).run()

In [104]:
# GPT 4
# Partial Run of Survey for Sample 1: Pre Question 19
result19_pre = q19_pre.by(sample1).by(m4).run()

In [105]:
# GPT 4
# Partial Run of Survey for Sample 1: Pre Question 20
result20_pre = q20_pre.by(sample1).by(m4).run()

In [106]:
# GPT 4
# Partial Run of Survey for Sample 1: Pre Question 22
result22_pre = q22_pre.by(sample1).by(m4).run()

#### Survey Run for Post-Information Shock for Sample 1: COMPLETE

In [128]:
# GPT 4
# Partial Run of Survey for Sample 1: Post Question 5
result5_post = q5_post.by(sample1).by(m4).run()

In [129]:
# GPT 4
# Partial Run of Survey for Sample 1: Post Question 6
result6_post = q6_post.by(sample1).by(m4).run()

In [157]:
# GPT 4
# Partial Run of Survey for Sample 1: Post Question 7
result7_post = q7_post.by(sample1).by(m4).run()

In [187]:
# GPT 4
# Partial Run of Survey for Sample 1: Post Question 8
result8_post = q8_post.by(sample1).by(m4).run()

In [39]:
# GPT 4
# Partial Run of Survey for Sample 1: Post Question 9
result9_post = q9_post.by(sample1).by(m4).run()

In [197]:
# GPT 4
# Partial Run of Survey for Sample 1: Post Question 17
result17_post = q17_post.by(sample1).by(m4).run()

In [209]:
# GPT 4
# Partial Run of Survey for Sample 1: Post Question 18
result18_post = q18_post.by(sample1).by(m4).run()

In [222]:
# GPT 4
# Partial Run of Survey for Sample 1: Post Question 19
result19_post = q19_post.by(sample1).by(m4).run()

In [232]:
# GPT 4
# Partial Run of Survey for Sample 1: Post Question 20
result20_post = q20_post.by(sample1).by(m4).run()

In [238]:
# GPT 4
# Partial Run of Survey for Sample 1: Post Question 22
result22_post = q22_post.by(sample1).by(m4).run()

<br>

---

<br>

### Run Survey for Sample 2:


#### Survey Run for AI Awareness and Use Questions for Sample 2:

In [28]:
# GPT 4
result2_1 = q1_cb.by(sample2).by(m4).run()

In [29]:
result2_2 = q2_cb.by(sample2).by(m4).run()

In [30]:
# GPT 3.5
result2_3 = q3_mc.by(sample2).by(m35).run()

In [31]:
# GPT 3.5
result2_4 = q4_ls.by(sample2).by(m35).run()

#### Survey Run for Demographic Questions for Sample 2:

In [32]:
# GPT 3.5
# Demographic Survey Run for Sample 2
result2_10 = q10_mc.by(sample2).by(m35).run()

In [33]:
result2_11 = q11_mc.by(sample2).by(m35).run()

In [34]:
result2_12 = q12_mc.by(sample2).by(m35).run()

In [35]:
result2_13 = q13_mc.by(sample2).by(m35).run()

In [36]:
result2_14 = q14_mc.by(sample2).by(m35).run()

In [37]:
result2_21 = q21_mc.by(sample2).by(m35).run()

#### Survey Run for Pre-Information Shock for Sample 2:

In [38]:
# GPT 4
# Survey Run for Pre Questions for Sample 2
result2_5_pre = q5_pre.by(sample2).by(m4).run()

In [39]:
result2_6_pre = q6_pre.by(sample2).by(m4).run()

In [40]:
result2_7_pre = q7_pre.by(sample2).by(m4).run()

In [41]:
result2_8_pre = q8_pre.by(sample2).by(m4).run()

In [42]:
result2_9_pre = q9_pre.by(sample2).by(m4).run()

In [43]:
result2_17_pre = q17_pre.by(sample2).by(m4).run()

In [44]:
result2_18_pre = q18_pre.by(sample2).by(m4).run()

In [45]:
result2_19_pre = q19_pre.by(sample2).by(m4).run()

In [46]:
result2_20_pre = q20_pre.by(sample2).by(m4).run()

In [47]:
result2_22_pre = q22_pre.by(sample2).by(m4).run()

#### Survey Run for Post-Information Shock for Sample 2:

In [48]:
# GPT 4
# Partial Survey Run for Post Questions for Sample 2
result2_5_post = q5_post.by(sample2).by(m4).run()

In [81]:
result2_6_post = q6_post.by(sample2).by(m4).run()

In [83]:
result2_7_post = q7_post.by(sample2).by(m4).run()

In [89]:
result2_8_post = q8_post.by(sample2).by(m4).run()

In [90]:
result2_9_post = q9_post.by(sample2).by(m4).run()


Task `post9` failed with `RateLimitError`:`Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-t5jMQ7PsiUZOTHgNK12M0T8X on tokens per day (TPD): Limit 1500000, Used 1499252, Requested 2887. Please try again in 2m3.206s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}`.

Task `post9` failed with `RateLimitError`:`Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-t5jMQ7PsiUZOTHgNK12M0T8X on tokens per day (TPD): Limit 1500000, Used 1499251, Requested 2896. Please try again in 2m3.667s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}`.

Task `post9` failed with `RateLimitError`:`Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-t5jMQ7PsiUZOTHgNK12M0T8X on to

In [184]:
result2_17_post = q17_post.by(sample2).by(m4).run()

In [185]:
result2_18_post = q18_post.by(sample2).by(m4).run()

In [186]:
result2_19_post = q19_post.by(sample2).by(m4).run()

In [187]:
result2_20_post = q20_post.by(sample2).by(m4).run()

In [92]:
result2_22_post = q22_post.by(sample2).by(m4).run()

<br>

---

<br>

### Run Survey for Sample 3:

#### Survey Run for AI Awareness and Use Questions for Sample 3:

In [93]:
# GPT 4
# Partial Survey Run for Sample 3
result3_1 = q1_cb.by(sample3).by(m4).run()

In [94]:
result3_2 = q2_cb.by(sample3).by(m4).run()

In [95]:
# GPT 3.5
# Partial Survey Run for Sample 3
result3_3 = q3_mc.by(sample3).by(m35).run()

In [96]:
result3_4 = q4_ls.by(sample3).by(m35).run()

#### Survey Run for Demographic Questions for Sample 3:

In [97]:
# GPT 3.5
# Demographic Survey Run for Sample 3
result3_10 = q10_mc.by(sample3).by(m35).run()

In [98]:
result3_11 = q11_mc.by(sample3).by(m35).run()

In [99]:
result3_12 = q12_mc.by(sample3).by(m35).run()

In [101]:
result3_13 = q13_mc.by(sample3).by(m35).run()

In [102]:
result3_14 = q14_mc.by(sample3).by(m35).run()

In [103]:
result3_21 = q21_mc.by(sample3).by(m35).run()

#### Survey Run for Pre-Information Shock for Sample 3:

In [260]:
# GPT 4
# Survey Run for Pre Questions for Sample 3
result3_5_pre = q5_pre.by(sample3).by(m4).run()

In [261]:
result3_6_pre = q6_pre.by(sample3).by(m4).run()

In [262]:
result3_7_pre = q7_pre.by(sample3).by(m4).run()

In [264]:
result3_8_pre = q8_pre.by(sample3).by(m4).run()

In [265]:
result3_9_pre = q9_pre.by(sample3).by(m4).run()

In [ ]:
result3_17_pre = q17_pre.by(sample3).by(m4).run()

In [ ]:
result3_18_pre = q18_pre.by(sample3).by(m4).run()

In [ ]:
result3_19_pre = q19_pre.by(sample3).by(m4).run()

In [ ]:
result3_20_pre = q20_pre.by(sample3).by(m4).run()

In [ ]:
result3_22_pre = q22_pre.by(sample3).by(m4).run()

#### Survey Run for Post-Information Shock for Sample 3:

In [ ]:
# GPT 4
# Survey Run for Post Questions for Sample 3
result3_5_post = q5_post.by(sample3).by(m4).run()

In [ ]:
result3_6_post = q6_post.by(sample3).by(m4).run()

In [ ]:
result3_7_post = q7_post.by(sample3).by(m4).run()

In [ ]:
result3_8_post = q8_post.by(sample3).by(m4).run()

In [ ]:
result3_9_post = q9_post.by(sample3).by(m4).run()

In [ ]:
result3_17_post = q17_post.by(sample3).by(m4).run()

In [ ]:
result3_18_post = q18_post.by(sample3).by(m4).run()

In [ ]:
result3_19_post = q19_post.by(sample3).by(m4).run()

In [ ]:
result3_20_post = q20_post.by(sample3).by(m4).run()

In [ ]:
result3_22_post = q22_post.by(sample3).by(m4).run()

<br>

---

<br>

## Run Survey for Sample 4:

#### Survey Run for AI Awareness and Use Questions for Sample 4:

In [ ]:
# GPT 4
# Partial Survey Run for Sample 4
result4_1 = q1_cb.by(sample4).by(m4).run()

In [ ]:
result4_2 = q2_cb.by(sample4).by(m4).run()

In [ ]:
# GPT 3.5
# Partial Survey Run for Sample 4
result4_3 = q3_mc.by(sample4).by(m35).run()

In [ ]:
result4_4 = q4_ls.by(sample4).by(m35).run()

#### Survey Run for Demographic Questions for Sample 4:

In [ ]:
# GPT 3.5
# Demographic Survey Run for Sample 4
result4_10 = q10_mc.by(sample4).by(m35).run()

In [ ]:
result4_11 = q11_mc.by(sample4).by(m35).run()

In [ ]:
result4_12 = q12_mc.by(sample4).by(m35).run()

In [ ]:
result4_13 = q13_mc.by(sample4).by(m35).run()

In [ ]:
result4_14 = q14_mc.by(sample4).by(m35).run()

In [ ]:
result4_21 = q21_mc.by(sample4).by(m35).run()

#### Survey Run for Pre-Information Shock for Sample 4:

In [ ]:
# GPT 4
# Survey Run for Pre Questions for Sample 4
result4_5_pre = q5_pre.by(sample4).by(m4).run()

In [ ]:
result4_6_pre = q6_pre.by(sample4).by(m4).run()

In [ ]:
result4_7_pre = q7_pre.by(sample4).by(m4).run()

In [ ]:
result4_8_pre = q8_pre.by(sample4).by(m4).run()

In [ ]:
result4_9_pre = q9_pre.by(sample4).by(m4).run()

In [ ]:
result4_17_pre = q17_pre.by(sample4).by(m4).run()

In [ ]:
result4_18_pre = q18_pre.by(sample4).by(m4).run()

In [ ]:
result4_19_pre = q19_pre.by(sample4).by(m4).run()

In [ ]:
result4_20_pre = q20_pre.by(sample4).by(m4).run()

In [ ]:
result4_22_pre = q22_pre.by(sample4).by(m4).run()

#### Survey Run for Post-Information Shock for Sample 4:

In [ ]:
# GPT 4
# Survey Run for Post Questions for Sample 4
result4_5_post = q5_post.by(sample4).by(m4).run()

In [ ]:
result4_6_post = q6_post.by(sample4).by(m4).run()

In [ ]:
result4_7_post = q7_post.by(sample4).by(m4).run()

In [ ]:
result4_8_post = q8_post.by(sample4).by(m4).run()

In [ ]:
result4_9_post = q9_post.by(sample4).by(m4).run()

In [ ]:
result4_17_post = q17_post.by(sample4).by(m4).run()

In [ ]:
result4_18_post = q18_post.by(sample4).by(m4).run()

In [ ]:
result4_19_post = q19_post.by(sample4).by(m4).run()

In [ ]:
result4_20_post = q20_post.by(sample4).by(m4).run()

In [ ]:
result4_22_post = q22_post.by(sample4).by(m4).run()

<br>

---

<br>

## Run Survey for Sample 5:

#### Survey Run for AI Awareness and Use Questions for Sample 5:

In [ ]:
# GPT 4
# Partial Survey Run for Sample 5
result5_1 = q1_cb.by(sample5).by(m4).run()

In [ ]:
result5_2 = q2_cb.by(sample5).by(m4).run()

In [ ]:
# GPT 3.5
# Partial Survey Run for Sample 5
result5_3 = q3_mc.by(sample5).by(m35).run()

In [ ]:
result5_4 = q4_ls.by(sample5).by(m35).run()

#### Survey Run for Demographic Questions for Sample 5:

In [ ]:
# GPT 3.5
# Demographic Survey Run for Sample 5
result5_10 = q10_mc.by(sample5).by(m35).run()

In [ ]:
result5_11 = q11_mc.by(sample5).by(m35).run()

In [ ]:
result5_12 = q12_mc.by(sample5).by(m35).run()

In [ ]:
result5_13 = q13_mc.by(sample5).by(m35).run()

In [ ]:
result5_14 = q14_mc.by(sample5).by(m35).run()

In [ ]:
result5_21 = q21_mc.by(sample5).by(m35).run()

#### Survey Run for Pre-Information Shock for Sample 5:

In [ ]:
# GPT 4
# Survey Run for Pre Questions for Sample 5
result5_5_pre = q5_pre.by(sample5).by(m4).run()

In [ ]:
result5_6_pre = q6_pre.by(sample5).by(m4).run()

In [ ]:
result5_7_pre = q7_pre.by(sample5).by(m4).run()

In [ ]:
result5_8_pre = q8_pre.by(sample5).by(m4).run()

In [ ]:
result5_9_pre = q9_pre.by(sample5).by(m4).run()

In [ ]:
result5_17_pre = q17_pre.by(sample5).by(m4).run()

In [ ]:
result5_18_pre = q18_pre.by(sample5).by(m4).run()

In [ ]:
result5_19_pre = q19_pre.by(sample5).by(m4).run()

In [ ]:
result5_20_pre = q20_pre.by(sample5).by(m4).run()

In [ ]:
result5_22_pre = q22_pre.by(sample5).by(m4).run()

#### Survey Run for Post-Information Shock for Sample 5:

In [ ]:
# GPT 4
# Survey Run for Post Questions for Sample 5
result5_5_post = q5_post.by(sample5).by(m4).run()

In [ ]:
result5_6_post = q6_post.by(sample5).by(m4).run()

In [ ]:
result5_7_post = q7_post.by(sample5).by(m4).run()

In [ ]:
result5_8_post = q8_post.by(sample5).by(m4).run()

In [ ]:
result5_9_post = q9_post.by(sample5).by(m4).run()

In [ ]:
result5_17_post = q17_post.by(sample5).by(m4).run()

In [ ]:
result5_18_post = q18_post.by(sample5).by(m4).run()

In [ ]:
result5_19_post = q19_post.by(sample5).by(m4).run()

In [ ]:
result5_20_post = q20_post.by(sample5).by(m4).run()

In [ ]:
result5_22_post = q22_post.by(sample5).by(m4).run()

<br>

---

<br>

## Run Survey for Sample 6:

#### Survey Run for AI Awareness and Use Questions for Sample 6:

In [ ]:
# GPT 4
# Partial Survey Run for Sample 6
result6_1 = q1_cb.by(sample6).by(m4).run()

In [ ]:
result6_2 = q2_cb.by(sample6).by(m4).run()

In [ ]:
# GPT 3.5
# Partial Survey Run for Sample 6
result6_3 = q3_mc.by(sample6).by(m35).run()

In [ ]:
result6_4 = q4_ls.by(sample6).by(m35).run()

#### Survey Run for Demographic Questions for Sample 6:

In [ ]:
# GPT 3.5
# Demographic Survey Run for Sample 6
result6_10 = q10_mc.by(sample6).by(m35).run()

In [ ]:
result6_11 = q11_mc.by(sample6).by(m35).run()

In [ ]:
result6_12 = q12_mc.by(sample6).by(m35).run()

In [ ]:
result6_13 = q13_mc.by(sample6).by(m35).run()

In [ ]:
result6_14 = q14_mc.by(sample6).by(m35).run()

In [ ]:
result6_21 = q21_mc.by(sample6).by(m35).run()

#### Survey Run for Pre-Information Shock for Sample 6:

In [ ]:
# GPT 4
# Survey Run for Pre Questions for Sample 6
result6_5_pre = q5_pre.by(sample6).by(m4).run()

In [ ]:
result6_6_pre = q6_pre.by(sample6).by(m4).run()

In [ ]:
result6_7_pre = q7_pre.by(sample6).by(m4).run()

In [ ]:
result6_8_pre = q8_pre.by(sample6).by(m4).run()

In [ ]:
result6_9_pre = q9_pre.by(sample6).by(m4).run()

In [ ]:
result6_17_pre = q17_pre.by(sample6).by(m4).run()

In [ ]:
result6_18_pre = q18_pre.by(sample6).by(m4).run()

In [ ]:
result6_19_pre = q19_pre.by(sample6).by(m4).run()

In [ ]:
result6_20_pre = q20_pre.by(sample6).by(m4).run()

In [ ]:
result6_22_pre = q22_pre.by(sample6).by(m4).run()

#### Survey Run for Post-Information Shock for Sample 6:

In [ ]:
# GPT 4
# Survey Run for Post Questions for Sample 6
result6_5_post = q5_post.by(sample6).by(m4).run()

In [ ]:
result6_6_post = q6_post.by(sample6).by(m4).run()

In [ ]:
result6_7_post = q7_post.by(sample6).by(m4).run()

In [ ]:
result6_8_post = q8_post.by(sample6).by(m4).run()

In [ ]:
result6_9_post = q9_post.by(sample6).by(m4).run()

In [ ]:
result6_17_post = q17_post.by(sample6).by(m4).run()

In [ ]:
result6_18_post = q18_post.by(sample6).by(m4).run()

In [ ]:
result6_19_post = q19_post.by(sample6).by(m4).run()

In [ ]:
result6_20_post = q20_post.by(sample6).by(m4).run()

In [ ]:
result6_22_post = q22_post.by(sample6).by(m4).run()

<br>

---

<br>

## Run Survey for Sample 7:

#### Survey Run for AI Awareness and Use Questions for Sample 7:

In [ ]:
# GPT 4
# Partial Survey Run for Sample 7
result7_1 = q1_cb.by(sample7).by(m4).run()

In [ ]:
result7_2 = q2_cb.by(sample7).by(m4).run()

In [ ]:
# GPT 3.5
# Partial Survey Run for Sample 7
result7_3 = q3_mc.by(sample7).by(m35).run()

In [ ]:
result7_4 = q4_ls.by(sample7).by(m35).run()

#### Survey Run for Demographic Questions for Sample 7:

In [ ]:
# GPT 3.5
# Demographic Survey Run for Sample 7
result7_10 = q10_mc.by(sample7).by(m35).run()

In [ ]:
result7_11 = q11_mc.by(sample7).by(m35).run()

In [ ]:
result7_12 = q12_mc.by(sample7).by(m35).run()

In [ ]:
result7_13 = q13_mc.by(sample7).by(m35).run()

In [ ]:
result7_14 = q14_mc.by(sample7).by(m35).run()

In [ ]:
result7_21 = q21_mc.by(sample7).by(m35).run()

#### Survey Run for Pre-Information Shock for Sample 7:

In [ ]:
# GPT 4
# Survey Run for Pre Questions for Sample 7
result7_5_pre = q5_pre.by(sample7).by(m4).run()

In [ ]:
result7_6_pre = q6_pre.by(sample7).by(m4).run()

In [ ]:
result7_7_pre = q7_pre.by(sample7).by(m4).run()

In [ ]:
result7_8_pre = q8_pre.by(sample7).by(m4).run()

In [ ]:
result7_9_pre = q9_pre.by(sample7).by(m4).run()

In [ ]:
result7_17_pre = q17_pre.by(sample7).by(m4).run()

In [ ]:
result7_18_pre = q18_pre.by(sample7).by(m4).run()

In [ ]:
result7_19_pre = q19_pre.by(sample7).by(m4).run()

In [ ]:
result7_20_pre = q20_pre.by(sample7).by(m4).run()

In [ ]:
result7_22_pre = q22_pre.by(sample7).by(m4).run()

#### Survey Run for Post-Information Shock for Sample 7:

In [ ]:
# GPT 4
# Survey Run for Post Questions for Sample 7
result7_5_post = q5_post.by(sample7).by(m4).run()

In [ ]:
result7_6_post = q6_post.by(sample7).by(m4).run()

In [ ]:
result7_7_post = q7_post.by(sample7).by(m4).run()

In [ ]:
result7_8_post = q8_post.by(sample7).by(m4).run()

In [ ]:
result7_9_post = q9_post.by(sample7).by(m4).run()

In [ ]:
result7_17_post = q17_post.by(sample7).by(m4).run()

In [ ]:
result7_18_post = q18_post.by(sample7).by(m4).run()

In [ ]:
result7_19_post = q19_post.by(sample7).by(m4).run()

In [ ]:
result7_20_post = q20_post.by(sample7).by(m4).run()

In [ ]:
result7_22_post = q22_post.by(sample7).by(m4).run()

<br>

---

<br>

## Run Survey for Sample 8:

#### Survey Run for AI Awareness and Use Questions for Sample 8:

In [ ]:
# GPT 4
# Partial Survey Run for Sample 8
result8_1 = q1_cb.by(sample8).by(m4).run()

In [ ]:
result8_2 = q2_cb.by(sample8).by(m4).run()

In [ ]:
# GPT 3.5
# Partial Survey Run for Sample 8
result8_3 = q3_mc.by(sample8).by(m35).run()

In [ ]:
result8_4 = q4_ls.by(sample8).by(m35).run()

#### Survey Run for Demographic Questions for Sample 8:

In [ ]:
# GPT 3.5
# Demographic Survey Run for Sample 8
result8_10 = q10_mc.by(sample8).by(m35).run()

In [ ]:
result8_11 = q11_mc.by(sample8).by(m35).run()

In [ ]:
result8_12 = q12_mc.by(sample8).by(m35).run()

In [ ]:
result8_13 = q13_mc.by(sample8).by(m35).run()

In [ ]:
result8_14 = q14_mc.by(sample8).by(m35).run()

In [ ]:
result8_21 = q21_mc.by(sample8).by(m35).run()

#### Survey Run for Pre-Information Shock for Sample 8:

In [ ]:
# GPT 4
# Survey Run for Pre Questions for Sample 8
result8_5_pre = q5_pre.by(sample8).by(m4).run()

In [ ]:
result8_6_pre = q6_pre.by(sample8).by(m4).run()

In [ ]:
result8_7_pre = q7_pre.by(sample8).by(m4).run()

In [ ]:
result8_8_pre = q8_pre.by(sample8).by(m4).run()

In [ ]:
result8_9_pre = q9_pre.by(sample8).by(m4).run()

In [ ]:
result8_17_pre = q17_pre.by(sample8).by(m4).run()

In [ ]:
result8_18_pre = q18_pre.by(sample8).by(m4).run()

In [ ]:
result8_19_pre = q19_pre.by(sample8).by(m4).run()

In [ ]:
result8_20_pre = q20_pre.by(sample8).by(m4).run()

In [ ]:
result8_22_pre = q22_pre.by(sample8).by(m4).run()

#### Survey Run for Post-Information Shock for Sample 8:

In [ ]:
# GPT 4
# Survey Run for Post Questions for Sample 8
result8_5_post = q5_post.by(sample8).by(m4).run()

In [ ]:
result8_6_post = q6_post.by(sample8).by(m4).run()

In [ ]:
result8_7_post = q7_post.by(sample8).by(m4).run()

In [ ]:
result8_8_post = q8_post.by(sample8).by(m4).run()

In [ ]:
result8_9_post = q9_post.by(sample8).by(m4).run()

In [ ]:
result8_17_post = q17_post.by(sample8).by(m4).run()

In [ ]:
result8_18_post = q18_post.by(sample8).by(m4).run()

In [ ]:
result8_19_post = q19_post.by(sample8).by(m4).run()

In [ ]:
result8_20_post = q20_post.by(sample8).by(m4).run()

In [ ]:
result8_22_post = q22_post.by(sample8).by(m4).run()

<br>

---

<br>

## Run Survey for Sample 9:

#### Survey Run for AI Awareness and Use Questions for Sample 9:

In [ ]:
# GPT 4
# Partial Survey Run for Sample 9
result9_1 = q1_cb.by(sample9).by(m4).run()

In [ ]:
result9_2 = q2_cb.by(sample9).by(m4).run()

In [ ]:
# GPT 3.5
# Partial Survey Run for Sample 9
result9_3 = q3_mc.by(sample9).by(m35).run()

In [ ]:
result9_4 = q4_ls.by(sample9).by(m35).run()

#### Survey Run for Demographic Questions for Sample 9:

In [ ]:
# GPT 3.5
# Demographic Survey Run for Sample 9
result9_10 = q10_mc.by(sample9).by(m35).run()

In [ ]:
result9_11 = q11_mc.by(sample9).by(m35).run()

In [ ]:
result9_12 = q12_mc.by(sample9).by(m35).run()

In [ ]:
result9_13 = q13_mc.by(sample9).by(m35).run()

In [ ]:
result9_14 = q14_mc.by(sample9).by(m35).run()

In [ ]:
result9_21 = q21_mc.by(sample9).by(m35).run()

#### Survey Run for Pre-Information Shock for Sample 9:

In [ ]:
# GPT 4
# Survey Run for Pre Questions for Sample 9
result9_5_pre = q5_pre.by(sample9).by(m4).run()

In [ ]:
result9_6_pre = q6_pre.by(sample9).by(m4).run()

In [ ]:
result9_7_pre = q7_pre.by(sample9).by(m4).run()

In [ ]:
result9_8_pre = q8_pre.by(sample9).by(m4).run()

In [ ]:
result9_9_pre = q9_pre.by(sample9).by(m4).run()

In [ ]:
result9_17_pre = q17_pre.by(sample9).by(m4).run()

In [ ]:
result9_18_pre = q18_pre.by(sample9).by(m4).run()

In [ ]:
result9_19_pre = q19_pre.by(sample9).by(m4).run()

In [ ]:
result9_20_pre = q20_pre.by(sample9).by(m4).run()

In [ ]:
result9_22_pre = q22_pre.by(sample9).by(m4).run()

#### Survey Run for Post-Information Shock for Sample 9:

In [ ]:
# GPT 4
# Survey Run for Post Questions for Sample 9
result9_5_post = q5_post.by(sample9).by(m4).run()

In [ ]:
result9_6_post = q6_post.by(sample9).by(m4).run()

In [ ]:
result9_7_post = q7_post.by(sample9).by(m4).run()

In [ ]:
result9_8_post = q8_post.by(sample9).by(m4).run()

In [ ]:
result9_9_post = q9_post.by(sample9).by(m4).run()

In [ ]:
result9_17_post = q17_post.by(sample9).by(m4).run()

In [ ]:
result9_18_post = q18_post.by(sample9).by(m4).run()

In [ ]:
result9_19_post = q19_post.by(sample9).by(m4).run()

In [ ]:
result9_20_post = q20_post.by(sample9).by(m4).run()

In [ ]:
result9_22_post = q22_post.by(sample9).by(m4).run()

<br>

---

<br>

## Run Survey for Sample 10:

#### Survey Run for AI Awareness and Use Questions for Sample 10:

In [ ]:
# GPT 4
# Partial Survey Run for Sample 10
result10_1 = q1_cb.by(sample10).by(m4).run()

In [ ]:
result10_2 = q2_cb.by(sample10).by(m4).run()

In [ ]:
# GPT 3.5
# Partial Survey Run for Sample 10
result10_3 = q3_mc.by(sample10).by(m35).run()

In [ ]:
result10_4 = q4_ls.by(sample10).by(m35).run()

#### Survey Run for Demographic Questions for Sample 10:

In [ ]:
# GPT 3.5
# Demographic Survey Run for Sample 10
result10_10 = q10_mc.by(sample10).by(m35).run()

In [ ]:
result10_11 = q11_mc.by(sample10).by(m35).run()

In [ ]:
result10_12 = q12_mc.by(sample10).by(m35).run()

In [ ]:
result10_13 = q13_mc.by(sample10).by(m35).run()

In [ ]:
result10_14 = q14_mc.by(sample10).by(m35).run()

In [ ]:
result10_21 = q21_mc.by(sample10).by(m35).run()

#### Survey Run for Pre-Information Shock for Sample 10:

In [ ]:
# GPT 4
# Survey Run for Pre Questions for Sample 10
result10_5_pre = q5_pre.by(sample10).by(m4).run()

In [ ]:
result10_6_pre = q6_pre.by(sample10).by(m4).run()

In [ ]:
result10_7_pre = q7_pre.by(sample10).by(m4).run()

In [ ]:
result10_8_pre = q8_pre.by(sample10).by(m4).run()

In [ ]:
result10_9_pre = q9_pre.by(sample10).by(m4).run()

In [ ]:
result10_17_pre = q17_pre.by(sample10).by(m4).run()

In [ ]:
result10_18_pre = q18_pre.by(sample10).by(m4).run()

In [ ]:
result10_19_pre = q19_pre.by(sample10).by(m4).run()

In [ ]:
result10_20_pre = q20_pre.by(sample10).by(m4).run()

In [ ]:
result10_22_pre = q22_pre.by(sample10).by(m4).run()

#### Survey Run for Post-Information Shock for Sample 10:

In [ ]:
# GPT 4
# Survey Run for Post Questions for Sample 10
result10_5_post = q5_post.by(sample10).by(m4).run()

In [ ]:
result10_6_post = q6_post.by(sample10).by(m4).run()

In [ ]:
result10_7_post = q7_post.by(sample10).by(m4).run()

In [ ]:
result10_8_post = q8_post.by(sample10).by(m4).run()

In [ ]:
result10_9_post = q9_post.by(sample10).by(m4).run()

In [ ]:
result10_17_post = q17_post.by(sample10).by(m4).run()

In [ ]:
result10_18_post = q18_post.by(sample10).by(m4).run()

In [ ]:
result10_19_post = q19_post.by(sample10).by(m4).run()

In [ ]:
result10_20_post = q20_post.by(sample10).by(m4).run()

In [ ]:
result10_22_post = q22_post.by(sample10).by(m4).run()

<br>

---

<br>


## Printing Survey Results

### Inspecting the Survey Results

<p>Here we inspect the results&mdash;more on how to do this in <a href="https://examples.goemeritus.com/example_visualization/"><i>visualization examples</i></a>.</p>

### Printing the Generated Trait Attributes for All 30 Simulated Agents in Sample 1:

In [ ]:
# Print all trait values for all 30 agents in sample 1
print(sample1[0])
print(sample1[1])
print(sample1[2])
print(sample1[3])
print(sample1[4])
print(sample1[5])
print(sample1[6])
print(sample1[7])
print(sample1[8])
print(sample1[9])
print(sample1[10])
print(sample1[11])
print(sample1[12])
print(sample1[13])
print(sample1[14])
print(sample1[15])
print(sample1[16])
print(sample1[17])
print(sample1[18])
print(sample1[19])
print(sample1[20])
print(sample1[21])
print(sample1[22])
print(sample1[23])
print(sample1[24])
print(sample1[25])
print(sample1[26])
print(sample1[27])
print(sample1[28])
print(sample1[29])

<br>

---

<br>

### Printing the Results of the Synthetic Survey Data with Pretty Labels for Sample 1:

In [239]:
# Sample 1 Survey Responses
# Pretty Labels

# Sample 1 Familiarity and Use Survey Question Responses
result1.select("q1_cb").print(pretty_labels={"answer.q1_cb":q1_cb.question_text})
result2.select("q2_cb").print(pretty_labels={"answer.q2_cb":q2_cb.question_text})
result3.select("q3_mc").print(pretty_labels={"answer.q3_mc":q3_mc.question_text})
result4.select("q4_ls").print(pretty_labels={"answer.q4_ls":q4_ls.question_text})

# Sample 1 Demographic Survey Question Responses
result10.select("q10_mc").print(pretty_labels={"answer.q10_mc":q10_mc.question_text})
result11.select("q11_mc").print(pretty_labels={"answer.q11_mc":q11_mc.question_text})
result12.select("q12_mc").print(pretty_labels={"answer.q12_mc":q12_mc.question_text})
result13.select("q13_mc").print(pretty_labels={"answer.q13_mc":q13_mc.question_text})
result14.select("q14_mc").print(pretty_labels={"answer.q14_mc":q14_mc.question_text})
result21.select("q21_mc").print(pretty_labels={"answer.q21_mc":q21_mc.question_text})

# Sample 1 Pre-Information Shock Survey Question Responses
result5_pre.select("pre5").print(pretty_labels={"answer.pre5":q5_pre.question_text})
result6_pre.select("pre6").print(pretty_labels={"answer.pre6":q6_pre.question_text})
result7_pre.select("pre7").print(pretty_labels={"answer.pre7":q7_pre.question_text})
result8_pre.select("pre8").print(pretty_labels={"answer.pre8":q8_pre.question_text})
result9_pre.select("pre9").print(pretty_labels={"answer.pre9":q9_pre.question_text})
result17_pre.select("pre17").print(pretty_labels={"answer.pre17":q17_pre.question_text})
result18_pre.select("pre18").print(pretty_labels={"answer.pre18":q18_pre.question_text})
result19_pre.select("pre19").print(pretty_labels={"answer.pre19":q19_pre.question_text})
result20_pre.select("pre20").print(pretty_labels={"answer.pre20":q20_pre.question_text})
result22_pre.select("pre22").print(pretty_labels={"answer.pre22":q22_pre.question_text})

# Sample 1 Post-Information Shock Survey Question Responses
result5_post.select("post5").print(pretty_labels={"answer.post5":q5_post.question_text})
result6_post.select("post6").print(pretty_labels={"answer.post6":q6_post.question_text})
result7_post.select("post7").print(pretty_labels={"answer.post7":q7_post.question_text})
result8_post.select("post8").print(pretty_labels={"answer.post8":q8_post.question_text})
result9_post.select("post9").print(pretty_labels={"answer.post9":q9_post.question_text})
result17_post.select("post17").print(pretty_labels={"answer.post17":q17_post.question_text})
result18_post.select("post18").print(pretty_labels={"answer.post18":q18_post.question_text})
result19_post.select("post19").print(pretty_labels={"answer.post19":q19_post.question_text})
result20_post.select("post20").print(pretty_labels={"answer.post20":q20_post.question_text})
result22_post.select("post22").print(pretty_labels={"answer.post22":q22_post.question_text})

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Which of the following AI products have you heard of before? Select all that apply                              ┃
┃ .                                                                                                               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ['ChatGPT', 'DALL-E', 'Whisper', 'Roomba', 'Siri', 'Alexa', 'Tesla', 'Netflix', 'Hulu', 'Google Maps', 'Waze',  │
│ 'Uber', 'Lyft', 'Midjourney', 'Microsoft Copilot', 'Character AI', 'Grammarly', 'Google Bard AI', 'Otter.ai']   │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'DALL-E', 'Roomba', 'Siri', 'Alexa', 'Tesla', 'Netflix', 'Hulu', 'Google Maps', 'Waze', 'Uber',     │
│ 'Lyft', 'Microsoft Copilot', 'Grammarly']                                                                       │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Alexa', 'Tesla', 'Netflix', 'Hulu', 'Google Maps', 'Waze', 'Uber', 'Lyft',       │
│ 'Grammarly']                                                                                                    │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'DALL-E', 'Whisper', 'Roomba', 'Siri', 'Alexa', 'Tesla', 'Netflix', 'Hulu', 'Google Maps', 'Waze',  │
│ 'Uber', 'Lyft', 'Midjourney', 'Microsoft Copilot', 'Character AI', 'Grammarly', 'Google Bard AI', 'Otter.ai']   │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Alexa', 'Tesla', 'Netflix', 'Hulu', 'Google Maps', 'Waze', 'Uber', 'Lyft']       │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'DALL-E', 'Roomba', 'Siri', 'Alexa', 'Tesla', 'Netflix', 'Hulu', 'Google Maps', 'Waze', 'Uber',     │
│ 'Lyft', 'Microsoft Copilot', 'Grammarly']                                                                       │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'DALL-E', 'Whisper', 'Roomba', 'Siri', 'Alexa', 'Tesla', 'Netflix', 'Hulu', 'Google Maps', 'Waze',  │
│ 'Uber', 'Lyft', 'Midjourney', 'Microsoft Copilot', 'Character AI', 'Grammarly', 'Google Bard AI', 'Otter.ai']   │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'DALL-E', 'Whisper', 'Roomba', 'Siri', 'Alexa', 'Tesla', 'Netflix', 'Hulu', 'Google Maps', 'Waze',  │
│ 'Uber', 'Lyft', 'Midjourney', 'Microsoft Copilot', 'Character AI', 'Grammarly', 'Google Bard AI', 'Otter.ai']   │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'DALL-E', 'Whisper', 'Roomba', 'Siri', 'Alexa', 'Tesla', 'Netflix', 'Hulu', 'Google Maps', 'Waze',  │
│ 'Uber', 'Lyft', 'Midjourney', 'Microsoft Copilot', 'Character AI', 'Grammarly', 'Google Bard AI', 'Otter.ai']   │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'DALL-E', 'Whisper', 'Roomba', 'Siri', 'Alexa', 'Tesla', 'Netflix', 'Hulu', 'Google Maps', 'Waze',  │
│ 'Uber', 'Lyft', 'Grammarly']                                                                                    │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Alexa', 'Tesla', 'Netflix', 'Google Maps', 'Waze', 'Uber', 'Lyft', 'Grammarly']  │
├───────────────────────────────────────────────────────

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Which of the following AI products have you used before? Select all that apply                        ┃
┃ .                                                                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ['ChatGPT', 'Roomba', 'Siri', 'Netflix', 'Google Maps', 'Uber', 'Grammarly']                          │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Alexa', 'Netflix', 'Google Maps', 'Uber', 'Grammarly']                 │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['Roomba', 'Siri', 'Netflix', 'Google Maps']                                                          │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Netflix', 'Google Maps', 'Uber', 'Grammarly']                          │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Alexa', 'Netflix', 'Hulu', 'Google Maps', 'Uber', 'Lyft', 'Grammarly'] │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Netflix', 'Google Maps', 'Uber', 'Grammarly']                          │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Alexa', 'Netflix', 'Google Maps', 'Uber', 'Grammarly']                 │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Alexa', 'Netflix', 'Google Maps', 'Uber', 'Grammarly']                 │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Netflix', 'Google Maps', 'Uber', 'Grammarly']                          │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Netflix', 'Google Maps', 'Uber', 'Grammarly']                          │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['Roomba', 'Siri', 'Netflix', 'Google Maps']                                                          │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Netflix', 'Google Maps', 'Grammarly']                                  │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Netflix', 'Google Maps', 'Uber', 'Grammarly']                          │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Netflix', 'Google Maps', 'Uber', 'Grammarly']                          │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Netflix', 'Google Maps', 'Uber', 'Grammarly']                          │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Netflix', 'Google Maps', 'Grammarly']                                  │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ['ChatGPT', 'Roomba', 'Siri', 'Netflix', 'Google Maps', 'Uber', 'Grammarly']                          │
├─────────────────────────────────────────────────────────────────────────────

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ How often did you use one or more AI products within the last month? ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Weekly                                                               │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Weekly                                                               │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Several days a week                                                  │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Weekly                                                               │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Weekly                                                               │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ Not at all                                                           │
├─────────────────────────────────────────────────────────

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ On a scale of 0-10, how much do you typically enjoy or benefit from current AI products? (0 = Not at all, 10 =  ┃
┃ Very much)                                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 2                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 0                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 7                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 4                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 5                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 0                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 7                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 0                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 5                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 3                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 2                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 5                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 5                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 4                                                                                                               │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 0                                                     

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Which of the following occupation groups does your job best fit in? ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Professionals                                                       │
├─────────────────────────────────────────────────────────────────────┤
│ Elementary Occupations                                              │
├─────────────────────────────────────────────────────────────────────┤
│ Craft and Related Trades Workers                                    │
├─────────────────────────────────────────────────────────────────────┤
│ Professionals                                                       │
├─────────────────────────────────────────────────────────────────────┤
│ Managers                                                            │
├─────────────────────────────────────────────────────────────────────┤
│ Craft and Related Trades Workers                                    │
├─────────────────────────────────────────────────────────────────────┤
│ Technicians                                                         │
├─────────────────────────────────────────────────────────────────────┤
│ Clerical Support Workers                                            │
├─────────────────────────────────────────────────────────────────────┤
│ Elementary Occupations                                              │
├─────────────────────────────────────────────────────────────────────┤
│ Technicians                                                         │
├─────────────────────────────────────────────────────────────────────┤
│ Elementary Occupations                                              │
├─────────────────────────────────────────────────────────────────────┤
│ Craft and Related Trades Workers                                    │
├─────────────────────────────────────────────────────────────────────┤
│ Services and Sales Workers                                          │
├─────────────────────────────────────────────────────────────────────┤
│ Technicians                                                         │
├─────────────────────────────────────────────────────────────────────┤
│ Machine Operators                                                   │
├─────────────────────────────────────────────────────────────────────┤
│ Other                                                               │
├─────────────────────────────────────────────────────────────────────┤
│ Technicians                                                         │
├─────────────────────────────────────────────────────────────────────┤
│ Services and Sales Workers                                          │
├─────────────────────────────────────────────────────────────────────┤
│ Technicians                                                         │
├─────────────────────────────────────────────────────────────────────┤
│ Services and Sales Workers                                          │
├─────────────────────────────────────────────────────────────────────┤
│ Clerical Support Workers                                            │
├─────────────────────────────────────────────────────────────────────┤
│ Other                                                               │
├─────────────────────────────────────────────────────────────────────┤
│ Other                                                               │
├─────────────────────────────────────────────────────────────────────┤
│ Machine Operators                                                   │
├─────────────────────────────────────────────────────────────────────┤
│ Services and Sales Workers                                          │
├─────────────────────────────────────────────────────────────────────┤
│ Managers                                                            │
├─────────────────────────────────────────────────────────────────────┤
│ Services and Sales Workers            

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International  ┃
┃ Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a    ┃
┃ substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined ┃
┃ to reflect the potential for AI to be integrated into each occupation based the tasks and skills that           ┃
┃ characterize each job                                                                                           ┃
┃ . Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology,   ┃
┃ the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing      ┃
┃ labor in the performance of a given task or occupation                                                          ┃
┃ .                                                                                                               ┃
┃ With respect to occupations, the research indicates that a large share of professional and managerial           ┃
┃ occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate          ┃
┃ exposure, followed by the US                                                                                    ┃
┃ . Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding      ┃
┃ figure for the US stands just above 15 percent                                                                  ┃
┃ . Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of           ┃
┃ complementarity, the difference between the US (14                                                              ┃
┃ .4 percent) and UK (10                                                                                          ┃
┃ .7 percent) remains relatively small                                                                            ┃
┃ . These occupational groups are also marked by high complementarity                                             ┃
┃ .                                                                                                               ┃
┃ With respect to gender differences, the research suggests women face a higher labor market exposure to AI       ┃
┃ compared to men                                                                                                 ┃
┃ . For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men   ┃
┃ . This outcome is primarily attributable to the occupational distribution across genders                        ┃
┃ . Female employment is more concentrated in service and retail occupations, which are relatively more exposed   ┃
┃ to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed         ┃
┃ . Despite women's higher labor market exposure to AI, focusing our attention to the potential for               ┃
┃ complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI     ┃
┃ . Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree   ┃
┃ of complementarity, the total shares of managers compared to professionals is relatively small                  ┃
┃ . Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in ┃
┃ the UK, US, Brazil, and South Africa                                                                            ┃
┃ . This vulnerability arises from their greater representation in clerical jobs, a category characterized by     ┃
┃ high exposure and low complementarity, thus at higher 

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International  ┃
┃ Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a    ┃
┃ substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined ┃
┃ to reflect the potential for AI to be integrated into each occupation based the tasks and skills that           ┃
┃ characterize each job                                                                                           ┃
┃ . Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology,   ┃
┃ the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing      ┃
┃ labor in the performance of a given task or occupation                                                          ┃
┃ .                                                                                                               ┃
┃ With respect to occupations, the research indicates that a large share of professional and managerial           ┃
┃ occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate          ┃
┃ exposure, followed by the US                                                                                    ┃
┃ . Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding      ┃
┃ figure for the US stands just above 15 percent                                                                  ┃
┃ . Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of           ┃
┃ complementarity, the difference between the US (14                                                              ┃
┃ .4 percent) and UK (10                                                                                          ┃
┃ .7 percent) remains relatively small                                                                            ┃
┃ . These occupational groups are also marked by high complementarity                                             ┃
┃ .                                                                                                               ┃
┃ With respect to gender differences, the research suggests women face a higher labor market exposure to AI       ┃
┃ compared to men                                                                                                 ┃
┃ . For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men   ┃
┃ . This outcome is primarily attributable to the occupational distribution across genders                        ┃
┃ . Female employment is more concentrated in service and retail occupations, which are relatively more exposed   ┃
┃ to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed         ┃
┃ . Despite women's higher labor market exposure to AI, focusing our attention to the potential for               ┃
┃ complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI     ┃
┃ . Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree   ┃
┃ of complementarity, the total shares of managers compared to professionals is relatively small                  ┃
┃ . Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in ┃
┃ the UK, US, Brazil, and South Africa                                                                            ┃
┃ . This vulnerability arises from their greater representation in clerical jobs, a category characterized by     ┃
┃ high exposure and low complementarity, thus at higher 

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International  ┃
┃ Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a    ┃
┃ substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined ┃
┃ to reflect the potential for AI to be integrated into each occupation based the tasks and skills that           ┃
┃ characterize each job                                                                                           ┃
┃ . Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology,   ┃
┃ the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing      ┃
┃ labor in the performance of a given task or occupation                                                          ┃
┃ .                                                                                                               ┃
┃ With respect to occupations, the research indicates that a large share of professional and managerial           ┃
┃ occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate          ┃
┃ exposure, followed by the US                                                                                    ┃
┃ . Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding      ┃
┃ figure for the US stands just above 15 percent                                                                  ┃
┃ . Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of           ┃
┃ complementarity, the difference between the US (14                                                              ┃
┃ .4 percent) and UK (10                                                                                          ┃
┃ .7 percent) remains relatively small                                                                            ┃
┃ . These occupational groups are also marked by high complementarity                                             ┃
┃ .                                                                                                               ┃
┃ With respect to gender differences, the research suggests women face a higher labor market exposure to AI       ┃
┃ compared to men                                                                                                 ┃
┃ . For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men   ┃
┃ . This outcome is primarily attributable to the occupational distribution across genders                        ┃
┃ . Female employment is more concentrated in service and retail occupations, which are relatively more exposed   ┃
┃ to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed         ┃
┃ . Despite women's higher labor market exposure to AI, focusing our attention to the potential for               ┃
┃ complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI     ┃
┃ . Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree   ┃
┃ of complementarity, the total shares of managers compared to professionals is relatively small                  ┃
┃ . Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in ┃
┃ the UK, US, Brazil, and South Africa                                                                            ┃
┃ . This vulnerability arises from their greater representation in clerical jobs, a category characterized by     ┃
┃ high exposure and low complementarity, thus at higher 

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International  ┃
┃ Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a    ┃
┃ substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined ┃
┃ to reflect the potential for AI to be integrated into each occupation based the tasks and skills that           ┃
┃ characterize each job                                                                                           ┃
┃ . Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology,   ┃
┃ the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing      ┃
┃ labor in the performance of a given task or occupation                                                          ┃
┃ .                                                                                                               ┃
┃ With respect to occupations, the research indicates that a large share of professional and managerial           ┃
┃ occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate          ┃
┃ exposure, followed by the US                                                                                    ┃
┃ . Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding      ┃
┃ figure for the US stands just above 15 percent                                                                  ┃
┃ . Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of           ┃
┃ complementarity, the difference between the US (14                                                              ┃
┃ .4 percent) and UK (10                                                                                          ┃
┃ .7 percent) remains relatively small                                                                            ┃
┃ . These occupational groups are also marked by high complementarity                                             ┃
┃ .                                                                                                               ┃
┃ With respect to gender differences, the research suggests women face a higher labor market exposure to AI       ┃
┃ compared to men                                                                                                 ┃
┃ . For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men   ┃
┃ . This outcome is primarily attributable to the occupational distribution across genders                        ┃
┃ . Female employment is more concentrated in service and retail occupations, which are relatively more exposed   ┃
┃ to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed         ┃
┃ . Despite women's higher labor market exposure to AI, focusing our attention to the potential for               ┃
┃ complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI     ┃
┃ . Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree   ┃
┃ of complementarity, the total shares of managers compared to professionals is relatively small                  ┃
┃ . Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in ┃
┃ the UK, US, Brazil, and South Africa                                                                            ┃
┃ . This vulnerability arises from their greater representation in clerical jobs, a category characterized by     ┃
┃ high exposure and low complementarity, thus at higher 

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ In the latest research on Labor Market Exposure to Artificial Intelligence (AI) conducted by the International  ┃
┃ Monterey Fund (IMF), 'complementarity' is defined as a measure of AI's potential as either a complement or a    ┃
┃ substitute for labor, where complementarity reflects lower risks of job displacement, and 'exposure' is defined ┃
┃ to reflect the potential for AI to be integrated into each occupation based the tasks and skills that           ┃
┃ characterize each job                                                                                           ┃
┃ . Given the high degree of uncertainty over the future of this fast-pacing and broadly applicable technology,   ┃
┃ the concept of exposure is purposely framed as agnostic on the likelihood of AI complementing or replacing      ┃
┃ labor in the performance of a given task or occupation                                                          ┃
┃ .                                                                                                               ┃
┃ With respect to occupations, the research indicates that a large share of professional and managerial           ┃
┃ occupations are almost entirely in the high-exposure categories, corroborating the UK's high aggregate          ┃
┃ exposure, followed by the US                                                                                    ┃
┃ . Nearly 30 percent of workers in the UK are employed in professional occupations, while the corresponding      ┃
┃ figure for the US stands just above 15 percent                                                                  ┃
┃ . Although the US hosts a larger share of managers, an occupation that exhibits the highest degree of           ┃
┃ complementarity, the difference between the US (14                                                              ┃
┃ .4 percent) and UK (10                                                                                          ┃
┃ .7 percent) remains relatively small                                                                            ┃
┃ . These occupational groups are also marked by high complementarity                                             ┃
┃ .                                                                                                               ┃
┃ With respect to gender differences, the research suggests women face a higher labor market exposure to AI       ┃
┃ compared to men                                                                                                 ┃
┃ . For instance, in the US, 68 percent of women are in high exposure occupations compared to 51 percent of men   ┃
┃ . This outcome is primarily attributable to the occupational distribution across genders                        ┃
┃ . Female employment is more concentrated in service and retail occupations, which are relatively more exposed   ┃
┃ to AI, while men are more likely to be in occupations intensive in manual labor, which are less exposed         ┃
┃ . Despite women's higher labor market exposure to AI, focusing our attention to the potential for               ┃
┃ complementarity suggests that women may have a higher likelihood of benefiting from the proliferation of AI     ┃
┃ . Conversely, although men generally exhibit a larger share of managerial jobs, which have the highest degree   ┃
┃ of complementarity, the total shares of managers compared to professionals is relatively small                  ┃
┃ . Nevertheless, women are also more susceptible to potential negative impacts from AI adoption, particularly in ┃
┃ the UK, US, Brazil, and South Africa                                                                            ┃
┃ . This vulnerability arises from their greater representation in clerical jobs, a category characterized by     ┃
┃ high exposure and low complementarity, thus at higher 

<br>

---

<br>

### Printing the Generated Trait Attributes for All 30 Simulated Agents in Sample 2:

In [ ]:
# Print all trait values for all 30 agents in sample 2
print(sample2[0])
print(sample2[1])
print(sample2[2])
print(sample2[3])
print(sample2[4])
print(sample2[5])
print(sample2[6])
print(sample2[7])
print(sample2[8])
print(sample2[9])
print(sample2[10])
print(sample2[11])
print(sample2[12])
print(sample2[13])
print(sample2[14])
print(sample2[15])
print(sample2[16])
print(sample2[17])
print(sample2[18])
print(sample2[19])
print(sample2[20])
print(sample2[21])
print(sample2[22])
print(sample2[23])
print(sample2[24])
print(sample2[25])
print(sample2[26])
print(sample2[27])
print(sample2[28])
print(sample2[29])

<br>

---

<br>

### Printing the Results of the Synthetic Survey Data with Pretty Labels for Sample 2:

In [ ]:
# Sample 2
# Pretty Labels

# Familiarity and Use Questions
result2_1.select("q1_cb").print(pretty_labels={"answer.q1_cb":q1_cb.question_text})
result2_2.select("q2_cb").print(pretty_labels={"answer.q2_cb":q2_cb.question_text})
result2_3.select("q3_mc").print(pretty_labels={"answer.q3_mc":q3_mc.question_text})
result2_4.select("q4_ls").print(pretty_labels={"answer.q4_ls":q4_ls.question_text})

# Demographic Questions
result2_10.select("q10_mc").print(pretty_labels={"answer.q10_mc":q10_mc.question_text})
result2_11.select("q11_mc").print(pretty_labels={"answer.q11_mc":q11_mc.question_text})
result2_12.select("q12_mc").print(pretty_labels={"answer.q12_mc":q12_mc.question_text})
result2_13.select("q13_mc").print(pretty_labels={"answer.q13_mc":q13_mc.question_text})
result2_14.select("q14_mc").print(pretty_labels={"answer.q14_mc":q14_mc.question_text})
result2_21.select("q21_mc").print(pretty_labels={"answer.q21_mc":q21_mc.question_text})

# Pre-Questions
result2_5_pre.select("pre5").print(pretty_labels={"answer.pre5":q5_pre.question_text})
result2_6_pre.select("pre6").print(pretty_labels={"answer.pre6":q6_pre.question_text})
result2_7_pre.select("pre7").print(pretty_labels={"answer.pre7":q7_pre.question_text})
result2_8_pre.select("pre8").print(pretty_labels={"answer.pre8":q8_pre.question_text})
result2_9_pre.select("pre9").print(pretty_labels={"answer.pre9":q9_pre.question_text})
result2_17_pre.select("pre17").print(pretty_labels={"answer.pre17":q17_pre.question_text})
result2_18_pre.select("pre18").print(pretty_labels={"answer.pre18":q18_pre.question_text})
result2_19_pre.select("pre19").print(pretty_labels={"answer.pre19":q19_pre.question_text})
result2_20_pre.select("pre20").print(pretty_labels={"answer.pre20":q20_pre.question_text})
result2_22_pre.select("pre22").print(pretty_labels={"answer.pre22":q22_pre.question_text})

# Post-Questions
result2_5_post.select("post5").print(pretty_labels={"answer.post5":q5_post.question_text})
result2_6_post.select("post6").print(pretty_labels={"answer.post6":q6_post.question_text})
result2_7_post.select("post7").print(pretty_labels={"answer.post7":q7_post.question_text})
result2_8_post.select("post8").print(pretty_labels={"answer.post8":q8_post.question_text})
result2_9_post.select("post9").print(pretty_labels={"answer.post9":q9_post.question_text})
result2_17_post.select("post17").print(pretty_labels={"answer.post17":q17_post.question_text})
result2_18_post.select("post18").print(pretty_labels={"answer.post18":q18_post.question_text})
result2_19_post.select("post19").print(pretty_labels={"answer.post19":q19_post.question_text})
result2_20_post.select("post20").print(pretty_labels={"answer.post20":q20_post.question_text})
result2_22_post.select("post22").print(pretty_labels={"answer.post22":q22_post.question_text})

<br>

---

<br>

### Printing the Generated Trait Attributes for All 30 Simulated Agents in Sample 3:

In [ ]:
# Print all trait values for all 30 agents in sample 3
print(sample3[0])
print(sample3[1])
print(sample3[2])
print(sample3[3])
print(sample3[4])
print(sample3[5])
print(sample3[6])
print(sample3[7])
print(sample3[8])
print(sample3[9])
print(sample3[10])
print(sample3[11])
print(sample3[12])
print(sample3[13])
print(sample3[14])
print(sample3[15])
print(sample3[16])
print(sample3[17])
print(sample3[18])
print(sample3[19])
print(sample3[20])
print(sample3[21])
print(sample3[22])
print(sample3[23])
print(sample3[24])
print(sample3[25])
print(sample3[26])
print(sample3[27])
print(sample3[28])
print(sample3[29])

<br>

---

<br>

### Printing the Results of the Synthetic Survey Data with Pretty Labels for Sample 3:

In [ ]:
# Sample 3
# Pretty Labels

# Familiarity and Use Questions
result3_1.select("q1_cb").print(pretty_labels={"answer.q1_cb":q1_cb.question_text})
result3_2.select("q2_cb").print(pretty_labels={"answer.q2_cb":q2_cb.question_text})
result3_3.select("q3_mc").print(pretty_labels={"answer.q3_mc":q3_mc.question_text})
result3_4.select("q4_ls").print(pretty_labels={"answer.q4_ls":q4_ls.question_text})

# Demographic Questions
result3_10.select("q10_mc").print(pretty_labels={"answer.q10_mc":q10_mc.question_text})
result3_11.select("q11_mc").print(pretty_labels={"answer.q11_mc":q11_mc.question_text})
result3_12.select("q12_mc").print(pretty_labels={"answer.q12_mc":q12_mc.question_text})
result3_13.select("q13_mc").print(pretty_labels={"answer.q13_mc":q13_mc.question_text})
result3_14.select("q14_mc").print(pretty_labels={"answer.q14_mc":q14_mc.question_text})
result3_21.select("q21_mc").print(pretty_labels={"answer.q21_mc":q21_mc.question_text})

# Pre-Questions
result3_5_pre.select("pre5").print(pretty_labels={"answer.pre5":q5_pre.question_text})
result3_6_pre.select("pre6").print(pretty_labels={"answer.pre6":q6_pre.question_text})
result3_7_pre.select("pre7").print(pretty_labels={"answer.pre7":q7_pre.question_text})
result3_8_pre.select("pre8").print(pretty_labels={"answer.pre8":q8_pre.question_text})
result3_9_pre.select("pre9").print(pretty_labels={"answer.pre9":q9_pre.question_text})
result3_17_pre.select("pre17").print(pretty_labels={"answer.pre17":q17_pre.question_text})
result3_18_pre.select("pre18").print(pretty_labels={"answer.pre18":q18_pre.question_text})
result3_19_pre.select("pre19").print(pretty_labels={"answer.pre19":q19_pre.question_text})
result3_20_pre.select("pre20").print(pretty_labels={"answer.pre20":q20_pre.question_text})
result3_22_pre.select("pre22").print(pretty_labels={"answer.pre22":q22_pre.question_text})

# Post-Questions
result3_5_post.select("post5").print(pretty_labels={"answer.post5":q5_post.question_text})
result3_6_post.select("post6").print(pretty_labels={"answer.post6":q6_post.question_text})
result3_7_post.select("post7").print(pretty_labels={"answer.post7":q7_post.question_text})
result3_8_post.select("post8").print(pretty_labels={"answer.post8":q8_post.question_text})
result3_9_post.select("post9").print(pretty_labels={"answer.post9":q9_post.question_text})
result3_17_post.select("post17").print(pretty_labels={"answer.post17":q17_post.question_text})
result3_18_post.select("post18").print(pretty_labels={"answer.post18":q18_post.question_text})
result3_19_post.select("post19").print(pretty_labels={"answer.post19":q19_post.question_text})
result3_20_post.select("post20").print(pretty_labels={"answer.post20":q20_post.question_text})
result3_22_post.select("post22").print(pretty_labels={"answer.post22":q22_post.question_text})

<br>

---

<br>

### Printing the Generated Trait Attributes for All 30 Simulated Agents in Sample 4:

In [ ]:
# Print all trait values for all 30 agents in sample 4
print(sample4[0])
print(sample4[1])
print(sample4[2])
print(sample4[3])
print(sample4[4])
print(sample4[5])
print(sample4[6])
print(sample4[7])
print(sample4[8])
print(sample4[9])
print(sample4[10])
print(sample4[11])
print(sample4[12])
print(sample4[13])
print(sample4[14])
print(sample4[15])
print(sample4[16])
print(sample4[17])
print(sample4[18])
print(sample4[19])
print(sample4[20])
print(sample4[21])
print(sample4[22])
print(sample4[23])
print(sample4[24])
print(sample4[25])
print(sample4[26])
print(sample4[27])
print(sample4[28])
print(sample4[29])

<br>

---

<br>

### Printing the Results of the Synthetic Survey Data with Pretty Labels for Sample 4:

In [ ]:
# Sample 4
# Pretty Labels

# Familiarity and Use Questions
result4_1.select("q1_cb").print(pretty_labels={"answer.q1_cb":q1_cb.question_text})
result4_2.select("q2_cb").print(pretty_labels={"answer.q2_cb":q2_cb.question_text})
result4_3.select("q3_mc").print(pretty_labels={"answer.q3_mc":q3_mc.question_text})
result4_4.select("q4_ls").print(pretty_labels={"answer.q4_ls":q4_ls.question_text})

# Demographic Questions
result4_10.select("q10_mc").print(pretty_labels={"answer.q10_mc":q10_mc.question_text})
result4_11.select("q11_mc").print(pretty_labels={"answer.q11_mc":q11_mc.question_text})
result4_12.select("q12_mc").print(pretty_labels={"answer.q12_mc":q12_mc.question_text})
result4_13.select("q13_mc").print(pretty_labels={"answer.q13_mc":q13_mc.question_text})
result4_14.select("q14_mc").print(pretty_labels={"answer.q14_mc":q14_mc.question_text})
result4_21.select("q21_mc").print(pretty_labels={"answer.q21_mc":q21_mc.question_text})

# Pre-Questions
result4_5_pre.select("pre5").print(pretty_labels={"answer.pre5":q5_pre.question_text})
result4_6_pre.select("pre6").print(pretty_labels={"answer.pre6":q6_pre.question_text})
result4_7_pre.select("pre7").print(pretty_labels={"answer.pre7":q7_pre.question_text})
result4_8_pre.select("pre8").print(pretty_labels={"answer.pre8":q8_pre.question_text})
result4_9_pre.select("pre9").print(pretty_labels={"answer.pre9":q9_pre.question_text})
result4_17_pre.select("pre17").print(pretty_labels={"answer.pre17":q17_pre.question_text})
result4_18_pre.select("pre18").print(pretty_labels={"answer.pre18":q18_pre.question_text})
result4_19_pre.select("pre19").print(pretty_labels={"answer.pre19":q19_pre.question_text})
result4_20_pre.select("pre20").print(pretty_labels={"answer.pre20":q20_pre.question_text})
result4_22_pre.select("pre22").print(pretty_labels={"answer.pre22":q22_pre.question_text})

# Post-Questions
result4_5_post.select("post5").print(pretty_labels={"answer.post5":q5_post.question_text})
result4_6_post.select("post6").print(pretty_labels={"answer.post6":q6_post.question_text})
result4_7_post.select("post7").print(pretty_labels={"answer.post7":q7_post.question_text})
result4_8_post.select("post8").print(pretty_labels={"answer.post8":q8_post.question_text})
result4_9_post.select("post9").print(pretty_labels={"answer.post9":q9_post.question_text})
result4_17_post.select("post17").print(pretty_labels={"answer.post17":q17_post.question_text})
result4_18_post.select("post18").print(pretty_labels={"answer.post18":q18_post.question_text})
result4_19_post.select("post19").print(pretty_labels={"answer.post19":q19_post.question_text})
result4_20_post.select("post20").print(pretty_labels={"answer.post20":q20_post.question_text})
result4_22_post.select("post22").print(pretty_labels={"answer.post22":q22_post.question_text})

<br>

---

<br>

### Printing the Generated Trait Attributes for All 30 Simulated Agents in Sample 5:

In [ ]:
# Print all trait values for all 30 agents in sample 5
print(sample5[0])
print(sample5[1])
print(sample5[2])
print(sample5[3])
print(sample5[4])
print(sample5[5])
print(sample5[6])
print(sample5[7])
print(sample5[8])
print(sample5[9])
print(sample5[10])
print(sample5[11])
print(sample5[12])
print(sample5[13])
print(sample5[14])
print(sample5[15])
print(sample5[16])
print(sample5[17])
print(sample5[18])
print(sample5[19])
print(sample5[20])
print(sample5[21])
print(sample5[22])
print(sample5[23])
print(sample5[24])
print(sample5[25])
print(sample5[26])
print(sample5[27])
print(sample5[28])
print(sample5[29])

<br>

---

<br>

### Printing the Results of the Synthetic Survey Data with Pretty Labels for Sample 5:

In [ ]:
# Sample 5
# Pretty Labels

# Familiarity and Use Questions
result5_1.select("q1_cb").print(pretty_labels={"answer.q1_cb":q1_cb.question_text})
result5_2.select("q2_cb").print(pretty_labels={"answer.q2_cb":q2_cb.question_text})
result5_3.select("q3_mc").print(pretty_labels={"answer.q3_mc":q3_mc.question_text})
result5_4.select("q4_ls").print(pretty_labels={"answer.q4_ls":q4_ls.question_text})

# Demographic Questions
result5_10.select("q10_mc").print(pretty_labels={"answer.q10_mc":q10_mc.question_text})
result5_11.select("q11_mc").print(pretty_labels={"answer.q11_mc":q11_mc.question_text})
result5_12.select("q12_mc").print(pretty_labels={"answer.q12_mc":q12_mc.question_text})
result5_13.select("q13_mc").print(pretty_labels={"answer.q13_mc":q13_mc.question_text})
result5_14.select("q14_mc").print(pretty_labels={"answer.q14_mc":q14_mc.question_text})
result5_21.select("q21_mc").print(pretty_labels={"answer.q21_mc":q21_mc.question_text})

# Pre-Questions
result5_5_pre.select("pre5").print(pretty_labels={"answer.pre5":q5_pre.question_text})
result5_6_pre.select("pre6").print(pretty_labels={"answer.pre6":q6_pre.question_text})
result5_7_pre.select("pre7").print(pretty_labels={"answer.pre7":q7_pre.question_text})
result5_8_pre.select("pre8").print(pretty_labels={"answer.pre8":q8_pre.question_text})
result5_9_pre.select("pre9").print(pretty_labels={"answer.pre9":q9_pre.question_text})
result5_17_pre.select("pre17").print(pretty_labels={"answer.pre17":q17_pre.question_text})
result5_18_pre.select("pre18").print(pretty_labels={"answer.pre18":q18_pre.question_text})
result5_19_pre.select("pre19").print(pretty_labels={"answer.pre19":q19_pre.question_text})
result5_20_pre.select("pre20").print(pretty_labels={"answer.pre20":q20_pre.question_text})
result5_22_pre.select("pre22").print(pretty_labels={"answer.pre22":q22_pre.question_text})

# Post-Questions
result5_5_post.select("post5").print(pretty_labels={"answer.post5":q5_post.question_text})
result5_6_post.select("post6").print(pretty_labels={"answer.post6":q6_post.question_text})
result5_7_post.select("post7").print(pretty_labels={"answer.post7":q7_post.question_text})
result5_8_post.select("post8").print(pretty_labels={"answer.post8":q8_post.question_text})
result5_9_post.select("post9").print(pretty_labels={"answer.post9":q9_post.question_text})
result5_17_post.select("post17").print(pretty_labels={"answer.post17":q17_post.question_text})
result5_18_post.select("post18").print(pretty_labels={"answer.post18":q18_post.question_text})
result5_19_post.select("post19").print(pretty_labels={"answer.post19":q19_post.question_text})
result5_20_post.select("post20").print(pretty_labels={"answer.post20":q20_post.question_text})
result5_22_post.select("post22").print(pretty_labels={"answer.post22":q22_post.question_text})

<br>

---

<br>

### Printing the Generated Trait Attributes for All 30 Simulated Agents in Sample 6:

In [ ]:
# Print all trait values for all 30 agents in sample 6
print(sample6[0])
print(sample6[1])
print(sample6[2])
print(sample6[3])
print(sample6[4])
print(sample6[5])
print(sample6[6])
print(sample6[7])
print(sample6[8])
print(sample6[9])
print(sample6[10])
print(sample6[11])
print(sample6[12])
print(sample6[13])
print(sample6[14])
print(sample6[15])
print(sample6[16])
print(sample6[17])
print(sample6[18])
print(sample6[19])
print(sample6[20])
print(sample6[21])
print(sample6[22])
print(sample6[23])
print(sample6[24])
print(sample6[25])
print(sample6[26])
print(sample6[27])
print(sample6[28])
print(sample6[29])

<br>

---

<br>

### Printing the Results of the Synthetic Survey Data with Pretty Labels for Sample 6:

In [ ]:
# Sample 6
# Pretty Labels

# Familiarity and Use Questions
result6_1.select("q1_cb").print(pretty_labels={"answer.q1_cb":q1_cb.question_text})
result6_2.select("q2_cb").print(pretty_labels={"answer.q2_cb":q2_cb.question_text})
result6_3.select("q3_mc").print(pretty_labels={"answer.q3_mc":q3_mc.question_text})
result6_4.select("q4_ls").print(pretty_labels={"answer.q4_ls":q4_ls.question_text})

# Demographic Questions
result6_10.select("q10_mc").print(pretty_labels={"answer.q10_mc":q10_mc.question_text})
result6_11.select("q11_mc").print(pretty_labels={"answer.q11_mc":q11_mc.question_text})
result6_12.select("q12_mc").print(pretty_labels={"answer.q12_mc":q12_mc.question_text})
result6_13.select("q13_mc").print(pretty_labels={"answer.q13_mc":q13_mc.question_text})
result6_14.select("q14_mc").print(pretty_labels={"answer.q14_mc":q14_mc.question_text})
result6_21.select("q21_mc").print(pretty_labels={"answer.q21_mc":q21_mc.question_text})

# Pre-Questions
result6_5_pre.select("pre5").print(pretty_labels={"answer.pre5":q5_pre.question_text})
result6_6_pre.select("pre6").print(pretty_labels={"answer.pre6":q6_pre.question_text})
result6_7_pre.select("pre7").print(pretty_labels={"answer.pre7":q7_pre.question_text})
result6_8_pre.select("pre8").print(pretty_labels={"answer.pre8":q8_pre.question_text})
result6_9_pre.select("pre9").print(pretty_labels={"answer.pre9":q9_pre.question_text})
result6_17_pre.select("pre17").print(pretty_labels={"answer.pre17":q17_pre.question_text})
result6_18_pre.select("pre18").print(pretty_labels={"answer.pre18":q18_pre.question_text})
result6_19_pre.select("pre19").print(pretty_labels={"answer.pre19":q19_pre.question_text})
result6_20_pre.select("pre20").print(pretty_labels={"answer.pre20":q20_pre.question_text})
result6_22_pre.select("pre22").print(pretty_labels={"answer.pre22":q22_pre.question_text})

# Post-Questions
result6_5_post.select("post5").print(pretty_labels={"answer.post5":q5_post.question_text})
result6_6_post.select("post6").print(pretty_labels={"answer.post6":q6_post.question_text})
result6_7_post.select("post7").print(pretty_labels={"answer.post7":q7_post.question_text})
result6_8_post.select("post8").print(pretty_labels={"answer.post8":q8_post.question_text})
result6_9_post.select("post9").print(pretty_labels={"answer.post9":q9_post.question_text})
result6_17_post.select("post17").print(pretty_labels={"answer.post17":q17_post.question_text})
result6_18_post.select("post18").print(pretty_labels={"answer.post18":q18_post.question_text})
result6_19_post.select("post19").print(pretty_labels={"answer.post19":q19_post.question_text})
result6_20_post.select("post20").print(pretty_labels={"answer.post20":q20_post.question_text})
result6_22_post.select("post22").print(pretty_labels={"answer.post22":q22_post.question_text})

<br>

---

<br>

### Printing the Generated Trait Attributes for All 30 Simulated Agents in Sample 7:

In [ ]:
# Print all trait values for all 30 agents in sample 7
print(sample7[0])
print(sample7[1])
print(sample7[2])
print(sample7[3])
print(sample7[4])
print(sample7[5])
print(sample7[6])
print(sample7[7])
print(sample7[8])
print(sample7[9])
print(sample7[10])
print(sample7[11])
print(sample7[12])
print(sample7[13])
print(sample7[14])
print(sample7[15])
print(sample7[16])
print(sample7[17])
print(sample7[18])
print(sample7[19])
print(sample7[20])
print(sample7[21])
print(sample7[22])
print(sample7[23])
print(sample7[24])
print(sample7[25])
print(sample7[26])
print(sample7[27])
print(sample7[28])
print(sample7[29])

<br>

---

<br>

### Printing the Results of the Synthetic Survey Data with Pretty Labels for Sample 7:

In [ ]:
# Sample 7
# Pretty Labels

# Familiarity and Use Questions
result7_1.select("q1_cb").print(pretty_labels={"answer.q1_cb":q1_cb.question_text})
result7_2.select("q2_cb").print(pretty_labels={"answer.q2_cb":q2_cb.question_text})
result7_3.select("q3_mc").print(pretty_labels={"answer.q3_mc":q3_mc.question_text})
result7_4.select("q4_ls").print(pretty_labels={"answer.q4_ls":q4_ls.question_text})

# Demographic Questions
result7_10.select("q10_mc").print(pretty_labels={"answer.q10_mc":q10_mc.question_text})
result7_11.select("q11_mc").print(pretty_labels={"answer.q11_mc":q11_mc.question_text})
result7_12.select("q12_mc").print(pretty_labels={"answer.q12_mc":q12_mc.question_text})
result7_13.select("q13_mc").print(pretty_labels={"answer.q13_mc":q13_mc.question_text})
result7_14.select("q14_mc").print(pretty_labels={"answer.q14_mc":q14_mc.question_text})
result7_21.select("q21_mc").print(pretty_labels={"answer.q21_mc":q21_mc.question_text})

# Pre-Questions
result7_5_pre.select("pre5").print(pretty_labels={"answer.pre5":q5_pre.question_text})
result7_6_pre.select("pre6").print(pretty_labels={"answer.pre6":q6_pre.question_text})
result7_7_pre.select("pre7").print(pretty_labels={"answer.pre7":q7_pre.question_text})
result7_8_pre.select("pre8").print(pretty_labels={"answer.pre8":q8_pre.question_text})
result7_9_pre.select("pre9").print(pretty_labels={"answer.pre9":q9_pre.question_text})
result7_17_pre.select("pre17").print(pretty_labels={"answer.pre17":q17_pre.question_text})
result7_18_pre.select("pre18").print(pretty_labels={"answer.pre18":q18_pre.question_text})
result7_19_pre.select("pre19").print(pretty_labels={"answer.pre19":q19_pre.question_text})
result7_20_pre.select("pre20").print(pretty_labels={"answer.pre20":q20_pre.question_text})
result7_22_pre.select("pre22").print(pretty_labels={"answer.pre22":q22_pre.question_text})

# Post-Questions
result7_5_post.select("post5").print(pretty_labels={"answer.post5":q5_post.question_text})
result7_6_post.select("post6").print(pretty_labels={"answer.post6":q6_post.question_text})
result7_7_post.select("post7").print(pretty_labels={"answer.post7":q7_post.question_text})
result7_8_post.select("post8").print(pretty_labels={"answer.post8":q8_post.question_text})
result7_9_post.select("post9").print(pretty_labels={"answer.post9":q9_post.question_text})
result7_17_post.select("post17").print(pretty_labels={"answer.post17":q17_post.question_text})
result7_18_post.select("post18").print(pretty_labels={"answer.post18":q18_post.question_text})
result7_19_post.select("post19").print(pretty_labels={"answer.post19":q19_post.question_text})
result7_20_post.select("post20").print(pretty_labels={"answer.post20":q20_post.question_text})
result7_22_post.select("post22").print(pretty_labels={"answer.post22":q22_post.question_text})

<br>

---

<br>

### Printing the Generated Trait Attributes for All 30 Simulated Agents in Sample 8:

In [ ]:
# Print all trait values for all 30 agents in sample 8
print(sample8[0])
print(sample8[1])
print(sample8[2])
print(sample8[3])
print(sample8[4])
print(sample8[5])
print(sample8[6])
print(sample8[7])
print(sample8[8])
print(sample8[9])
print(sample8[10])
print(sample8[11])
print(sample8[12])
print(sample8[13])
print(sample8[14])
print(sample8[15])
print(sample8[16])
print(sample8[17])
print(sample8[18])
print(sample8[19])
print(sample8[20])
print(sample8[21])
print(sample8[22])
print(sample8[23])
print(sample8[24])
print(sample8[25])
print(sample8[26])
print(sample8[27])
print(sample8[28])
print(sample8[29])

<br>

---

<br>

### Printing the Results of the Synthetic Survey Data with Pretty Labels for Sample 8:

In [ ]:
# Sample 8
# Pretty Labels

# Familiarity and Use Questions
result8_1.select("q1_cb").print(pretty_labels={"answer.q1_cb":q1_cb.question_text})
result8_2.select("q2_cb").print(pretty_labels={"answer.q2_cb":q2_cb.question_text})
result8_3.select("q3_mc").print(pretty_labels={"answer.q3_mc":q3_mc.question_text})
result8_4.select("q4_ls").print(pretty_labels={"answer.q4_ls":q4_ls.question_text})

# Demographic Questions
result8_10.select("q10_mc").print(pretty_labels={"answer.q10_mc":q10_mc.question_text})
result8_11.select("q11_mc").print(pretty_labels={"answer.q11_mc":q11_mc.question_text})
result8_12.select("q12_mc").print(pretty_labels={"answer.q12_mc":q12_mc.question_text})
result8_13.select("q13_mc").print(pretty_labels={"answer.q13_mc":q13_mc.question_text})
result8_14.select("q14_mc").print(pretty_labels={"answer.q14_mc":q14_mc.question_text})
result8_21.select("q21_mc").print(pretty_labels={"answer.q21_mc":q21_mc.question_text})

# Pre-Questions
result8_5_pre.select("pre5").print(pretty_labels={"answer.pre5":q5_pre.question_text})
result8_6_pre.select("pre6").print(pretty_labels={"answer.pre6":q6_pre.question_text})
result8_7_pre.select("pre7").print(pretty_labels={"answer.pre7":q7_pre.question_text})
result8_8_pre.select("pre8").print(pretty_labels={"answer.pre8":q8_pre.question_text})
result8_9_pre.select("pre9").print(pretty_labels={"answer.pre9":q9_pre.question_text})
result8_17_pre.select("pre17").print(pretty_labels={"answer.pre17":q17_pre.question_text})
result8_18_pre.select("pre18").print(pretty_labels={"answer.pre18":q18_pre.question_text})
result8_19_pre.select("pre19").print(pretty_labels={"answer.pre19":q19_pre.question_text})
result8_20_pre.select("pre20").print(pretty_labels={"answer.pre20":q20_pre.question_text})
result8_22_pre.select("pre22").print(pretty_labels={"answer.pre22":q22_pre.question_text})

# Post-Questions
result8_5_post.select("post5").print(pretty_labels={"answer.post5":q5_post.question_text})
result8_6_post.select("post6").print(pretty_labels={"answer.post6":q6_post.question_text})
result8_7_post.select("post7").print(pretty_labels={"answer.post7":q7_post.question_text})
result8_8_post.select("post8").print(pretty_labels={"answer.post8":q8_post.question_text})
result8_9_post.select("post9").print(pretty_labels={"answer.post9":q9_post.question_text})
result8_17_post.select("post17").print(pretty_labels={"answer.post17":q17_post.question_text})
result8_18_post.select("post18").print(pretty_labels={"answer.post18":q18_post.question_text})
result8_19_post.select("post19").print(pretty_labels={"answer.post19":q19_post.question_text})
result8_20_post.select("post20").print(pretty_labels={"answer.post20":q20_post.question_text})
result8_22_post.select("post22").print(pretty_labels={"answer.post22":q22_post.question_text})

<br>

---

<br>

### Generating Trait Attributes for All 30 Agents in Sample 9:

In [ ]:
# Print all trait values for all 30 agents in sample 9
print(sample9[0])
print(sample9[1])
print(sample9[2])
print(sample9[3])
print(sample9[4])
print(sample9[5])
print(sample9[6])
print(sample9[7])
print(sample9[8])
print(sample9[9])
print(sample9[10])
print(sample9[11])
print(sample9[12])
print(sample9[13])
print(sample9[14])
print(sample9[15])
print(sample9[16])
print(sample9[17])
print(sample9[18])
print(sample9[19])
print(sample9[20])
print(sample9[21])
print(sample9[22])
print(sample9[23])
print(sample9[24])
print(sample9[25])
print(sample9[26])
print(sample9[27])
print(sample9[28])
print(sample9[29])

<br>

---

<br>

### Printing the Results of the Synthetic Survey Data with Pretty Labels for Sample 9:

In [ ]:
# Sample 9
# Pretty Labels

# Familiarity and Use Questions
result9_1.select("q1_cb").print(pretty_labels={"answer.q1_cb":q1_cb.question_text})
result9_2.select("q2_cb").print(pretty_labels={"answer.q2_cb":q2_cb.question_text})
result9_3.select("q3_mc").print(pretty_labels={"answer.q3_mc":q3_mc.question_text})
result9_4.select("q4_ls").print(pretty_labels={"answer.q4_ls":q4_ls.question_text})

# Demographic Questions
result9_10.select("q10_mc").print(pretty_labels={"answer.q10_mc":q10_mc.question_text})
result9_11.select("q11_mc").print(pretty_labels={"answer.q11_mc":q11_mc.question_text})
result9_12.select("q12_mc").print(pretty_labels={"answer.q12_mc":q12_mc.question_text})
result9_13.select("q13_mc").print(pretty_labels={"answer.q13_mc":q13_mc.question_text})
result9_14.select("q14_mc").print(pretty_labels={"answer.q14_mc":q14_mc.question_text})
result9_21.select("q21_mc").print(pretty_labels={"answer.q21_mc":q21_mc.question_text})

# Pre-Questions
result9_5_pre.select("pre5").print(pretty_labels={"answer.pre5":q5_pre.question_text})
result9_6_pre.select("pre6").print(pretty_labels={"answer.pre6":q6_pre.question_text})
result9_7_pre.select("pre7").print(pretty_labels={"answer.pre7":q7_pre.question_text})
result9_8_pre.select("pre8").print(pretty_labels={"answer.pre8":q8_pre.question_text})
result9_9_pre.select("pre9").print(pretty_labels={"answer.pre9":q9_pre.question_text})
result9_17_pre.select("pre17").print(pretty_labels={"answer.pre17":q17_pre.question_text})
result9_18_pre.select("pre18").print(pretty_labels={"answer.pre18":q18_pre.question_text})
result9_19_pre.select("pre19").print(pretty_labels={"answer.pre19":q19_pre.question_text})
result9_20_pre.select("pre20").print(pretty_labels={"answer.pre20":q20_pre.question_text})
result9_22_pre.select("pre22").print(pretty_labels={"answer.pre22":q22_pre.question_text})

# Post-Questions
result9_5_post.select("post5").print(pretty_labels={"answer.post5":q5_post.question_text})
result9_6_post.select("post6").print(pretty_labels={"answer.post6":q6_post.question_text})
result9_7_post.select("post7").print(pretty_labels={"answer.post7":q7_post.question_text})
result9_8_post.select("post8").print(pretty_labels={"answer.post8":q8_post.question_text})
result9_9_post.select("post9").print(pretty_labels={"answer.post9":q9_post.question_text})
result9_17_post.select("post17").print(pretty_labels={"answer.post17":q17_post.question_text})
result9_18_post.select("post18").print(pretty_labels={"answer.post18":q18_post.question_text})
result9_19_post.select("post19").print(pretty_labels={"answer.post19":q19_post.question_text})
result9_20_post.select("post20").print(pretty_labels={"answer.post20":q20_post.question_text})
result9_22_post.select("post22").print(pretty_labels={"answer.post22":q22_post.question_text})

<br>

---

<br>

### Printing the Generated Trait Attributes for All 30 Simulated Agents in Sample 10:

In [ ]:
# Print all trait values for all 30 agents in sample 10
print(sample10[0])
print(sample10[1])
print(sample10[2])
print(sample10[3])
print(sample10[4])
print(sample10[5])
print(sample10[6])
print(sample10[7])
print(sample10[8])
print(sample10[9])
print(sample10[10])
print(sample10[11])
print(sample10[12])
print(sample10[13])
print(sample10[14])
print(sample10[15])
print(sample10[16])
print(sample10[17])
print(sample10[18])
print(sample10[19])
print(sample10[20])
print(sample10[21])
print(sample10[22])
print(sample10[23])
print(sample10[24])
print(sample10[25])
print(sample10[26])
print(sample10[27])
print(sample10[28])
print(sample10[29])

<br>

---

<br>

### Printing the Results of the Synthetic Survey Data with Pretty Labels for Sample 10:

In [ ]:
# Sample 10
# Pretty Labels

# Familiarity and Use Questions
result10_1.select("q1_cb").print(pretty_labels={"answer.q1_cb":q1_cb.question_text})
result10_2.select("q2_cb").print(pretty_labels={"answer.q2_cb":q2_cb.question_text})
result10_3.select("q3_mc").print(pretty_labels={"answer.q3_mc":q3_mc.question_text})
result10_4.select("q4_ls").print(pretty_labels={"answer.q4_ls":q4_ls.question_text})

# Demographic Questions
result10_10.select("q10_mc").print(pretty_labels={"answer.q10_mc":q10_mc.question_text})
result10_11.select("q11_mc").print(pretty_labels={"answer.q11_mc":q11_mc.question_text})
result10_12.select("q12_mc").print(pretty_labels={"answer.q12_mc":q12_mc.question_text})
result10_13.select("q13_mc").print(pretty_labels={"answer.q13_mc":q13_mc.question_text})
result10_14.select("q14_mc").print(pretty_labels={"answer.q14_mc":q14_mc.question_text})
result10_21.select("q21_mc").print(pretty_labels={"answer.q21_mc":q21_mc.question_text})

# Pre-Questions
result10_5_pre.select("pre5").print(pretty_labels={"answer.pre5":q5_pre.question_text})
result10_6_pre.select("pre6").print(pretty_labels={"answer.pre6":q6_pre.question_text})
result10_7_pre.select("pre7").print(pretty_labels={"answer.pre7":q7_pre.question_text})
result10_8_pre.select("pre8").print(pretty_labels={"answer.pre8":q8_pre.question_text})
result10_9_pre.select("pre9").print(pretty_labels={"answer.pre9":q9_pre.question_text})
result10_17_pre.select("pre17").print(pretty_labels={"answer.pre17":q17_pre.question_text})
result10_18_pre.select("pre18").print(pretty_labels={"answer.pre18":q18_pre.question_text})
result10_19_pre.select("pre19").print(pretty_labels={"answer.pre19":q19_pre.question_text})
result10_20_pre.select("pre20").print(pretty_labels={"answer.pre20":q20_pre.question_text})
result10_22_pre.select("pre22").print(pretty_labels={"answer.pre22":q22_pre.question_text})

# Post-Questions
result10_5_post.select("post5").print(pretty_labels={"answer.post5":q5_post.question_text})
result10_6_post.select("post6").print(pretty_labels={"answer.post6":q6_post.question_text})
result10_7_post.select("post7").print(pretty_labels={"answer.post7":q7_post.question_text})
result10_8_post.select("post8").print(pretty_labels={"answer.post8":q8_post.question_text})
result10_9_post.select("post9").print(pretty_labels={"answer.post9":q9_post.question_text})
result10_17_post.select("post17").print(pretty_labels={"answer.post17":q17_post.question_text})
result10_18_post.select("post18").print(pretty_labels={"answer.post18":q18_post.question_text})
result10_19_post.select("post19").print(pretty_labels={"answer.post19":q19_post.question_text})
result10_20_post.select("post20").print(pretty_labels={"answer.post20":q20_post.question_text})
result10_22_post.select("post22").print(pretty_labels={"answer.post22":q22_post.question_text})

<br>

---

<br>

## Data Cleaning

In [155]:
import json
import pandas as pd
import numpy as np

In [142]:
# Creating a DataFrame from the agents list
agents_data = pd.DataFrame([agent.traits for agent in agents])

### Sample 1: Creating Dataframe for Agent Traits and Survey Responses

In [ ]:
result1.to_pandas().columns

In [ ]:
result1_df = result1.to_pandas()[['agent.personas', 'agent.doomers', 'agent.age', 'agent.gender', 'agent.education',
	'agent.occupation', 'agent.income', 'agent.nationality_politics', 'agent.politics', 'agent.economics',
	'answer.q1_cb']]

In [ ]:
result2_df = result2.to_pandas()[['answer.q2_cb']]
result3_df = result3.to_pandas()[['answer.q3_mc']]
result4_df = result4.to_pandas()[['answer.q4_ls']]

result10_df = result10.to_pandas()[['answer.q10_mc']]
result11_df = result11.to_pandas()[['answer.q11_mc']]
result12_df = result12.to_pandas()[['answer.q12_mc']]
result13_df = result13.to_pandas()[['answer.q13_mc']]
result14_df = result14.to_pandas()[['answer.q14_mc']]
result21_df = result21.to_pandas()[['answer.q21_mc']]

result5_pre_df = result5_pre.to_pandas()[['answer.pre5']]
result6_pre_df = result6_pre.to_pandas()[['answer.pre6']]
result7_pre_df = result7_pre.to_pandas()[['answer.pre7']]
result8_pre_df = result8_pre.to_pandas()[['answer.pre8']]
result9_pre_df = result9_pre.to_pandas()[['answer.pre9']]

result17_pre_df = result17_pre.to_pandas()[['answer.pre17']]
result18_pre_df = result18_pre.to_pandas()[['answer.pre18']]
result19_pre_df = result19_pre.to_pandas()[['answer.pre19']]
result20_pre_df = result20_pre.to_pandas()[['answer.pre20']]
result22_pre_df = result22_pre.to_pandas()[['answer.pre22']]

result5_post_df = result5_post.to_pandas()[['answer.post5']]
result6_post_df = result6_post.to_pandas()[['answer.post6']]
result7_post_df = result7_post.to_pandas()[['answer.post7']]
result8_post_df = result8_post.to_pandas()[['answer.post8']]
result9_post_df = result9_post.to_pandas()[['answer.post9']]

result17_post_df = result17_post.to_pandas()[['answer.post17']]
result18_post_df = result18_post.to_pandas()[['answer.post18']]
result19_post_df = result19_post.to_pandas()[['answer.post19']]
result20_post_df = result20_post.to_pandas()[['answer.post20']]
result22_post_df = result22_post.to_pandas()[['answer.post22']]

In [ ]:
# Concatenate dataframess column-wise
sample1_df = pd.concat([result1_df, result2_df, result3_df, result4_df, result10_df, result11_df, result12_df, result13_df, result14_df, result21_df, result5_pre_df, result6_pre_df, result7_pre_df, result8_pre_df, result9_pre_df, result17_pre_df, result18_pre_df, result19_pre_df, result20_pre_df, result22_pre_df, result5_post_df, result6_post_df, result7_post_df, result8_post_df, result9_post_df, result17_post_df, result18_post_df, result19_post_df, result20_post_df, result22_post_df], axis=1)

In [ ]:
# Rename the columns
sample1_df = sample1_df.rename(columns={"personas": "Persona", "doomers": "Doomer Exposure", "age": "Age", "gender":"Gender", "education": "Educational Attainment", "income":"Annual Pre-Tax Income", "occupation": "Occupation", "nationality_politics": "Nationality and US Political Party Preference", "politics": "Political Orientation", "economics": "Economic Views"})

# Add the 'Sample' column with all values set to 1
sample1_df['Sample'] = 1

# Add the 'Agent' column, which is the index + 1
sample1_df['Agent'] = sample1_df.index + 1

# Reorder the DataFrame to have 'Sample' and 'Agent' as the first two columns
column_order = ['Sample', 'Agent'] + [col for col in sample1_df.columns if col not in ['Sample', 'Agent']]
sample1_df = sample1_df[column_order]

# Display the first few rows of the modified DataFrame to verify
print(sample1_df.head())

In [ ]:
# Export the DataFrame to a CSV file
sample1_df.to_csv('sample1.csv', index=False)

<br>

---

<br>

### Sample 2: Creating Dataframe for Agent Traits and Survey Responses

In [86]:
result2_1.to_pandas().columns

Index(['answer.q1_cb_comment', 'agent.economics', 'agent.income',
       'model.max_tokens', 'model.temperature', 'agent.nationality_politics',
       'model.model', 'agent.gender', 'prompt.q1_cb_system_prompt',
       'agent.age', 'answer.q1_cb', 'model.presence_penalty', 'agent.doomers',
       'agent.education', 'model.use_cache', 'agent.personas', 'model.top_p',
       'model.frequency_penalty', 'agent.occupation', 'agent.politics',
       'prompt.q1_cb_user_prompt'],
      dtype='object')

In [75]:
result2_1_df = result2_1.to_pandas()[['agent.personas', 'agent.doomers', 'agent.age', 'agent.gender', 'agent.education',
	'agent.occupation', 'agent.income', 'agent.nationality_politics', 'agent.politics', 'agent.economics',
	'answer.q1_cb']]

In [ ]:
result2_2_df = result2_2.to_pandas()[['answer.q2_cb']]
result2_3_df = result2_3.to_pandas()[['answer.q3_mc']]
result2_4_df = result2_4.to_pandas()[['answer.q4_ls']]

result2_10_df = result2_10.to_pandas()[['answer.q10_mc']]
result2_11_df = result2_11.to_pandas()[['answer.q11_mc']]
result2_12_df = result2_12.to_pandas()[['answer.q12_mc']]
result2_13_df = result2_13.to_pandas()[['answer.q13_mc']]
result2_14_df = result2_14.to_pandas()[['answer.q14_mc']]
result2_21_df = result2_21.to_pandas()[['answer.q21_mc']]

result2_5_pre_df = result2_5_pre.to_pandas()[['answer.pre5']]
result2_6_pre_df = result2_6_pre.to_pandas()[['answer.pre6']]
result2_7_pre_df = result2_7_pre.to_pandas()[['answer.pre7']]
result2_8_pre_df = result2_8_pre.to_pandas()[['answer.pre8']]
result2_9_pre_df = result2_9_pre.to_pandas()[['answer.pre9']]

result2_17_pre_df = result2_17_pre.to_pandas()[['answer.pre17']]
result2_18_pre_df = result2_18_pre.to_pandas()[['answer.pre18']]
result2_19_pre_df = result2_19_pre.to_pandas()[['answer.pre19']]
result2_20_pre_df = result2_20_pre.to_pandas()[['answer.pre20']]
result2_22_pre_df = result2_22_pre.to_pandas()[['answer.pre22']]

result2_5_post_df = result2_5_post.to_pandas()[['answer.post5']]
result2_6_post_df = result2_6_post.to_pandas()[['answer.post6']]
result2_7_post_df = result2_7_post.to_pandas()[['answer.post7']]
result2_8_post_df = result2_8_post.to_pandas()[['answer.post8']]
result2_9_post_df = result2_9_post.to_pandas()[['answer.post9']]

result2_17_post_df = result2_17_post.to_pandas()[['answer.post17']]
result2_18_post_df = result2_18_post.to_pandas()[['answer.post18']]
result2_19_post_df = result2_19_post.to_pandas()[['answer.post19']]
result2_20_post_df = result2_20_post.to_pandas()[['answer.post20']]
result2_22_post_df = result2_22_post.to_pandas()[['answer.post22']]

In [ ]:
# Concatenate dataframess column-wise
sample2_df = pd.concat([result2_1_df, result2_2_df, result2_3_df, result2_4_df, result2_10_df, result2_11_df, result2_12_df, result2_13_df, result2_14_df, result2_21_df, result2_5_pre_df, result2_6_pre_df, result2_7_pre_df, result2_8_pre_df, result2_9_pre_df, result2_17_pre_df, result2_18_pre_df, result2_19_pre_df, result2_20_pre_df, result2_22_pre_df, result2_5_post_df, result2_6_post_df, result2_7_post_df, result2_8_post_df, result2_9_post_df, result2_17_post_df, result2_18_post_df, result2_19_post_df, result2_20_post_df, result2_22_post_df], axis=1)

In [ ]:
# Rename the columns
sample2_df = sample2_df.rename(columns={"personas": "Persona", "doomers": "Doomer Exposure", "age": "Age", "gender":"Gender", "education": "Educational Attainment", "income":"Annual Pre-Tax Income", "occupation": "Occupation", "nationality_politics": "Nationality and US Political Party Preference", "politics": "Political Orientation", "economics": "Economic Views"})

# Add the 'Sample' column with all values set to 2
sample2_df['Sample'] = 2

# Add the 'Agent' column, which is the index + 1
sample2_df['Agent'] = sample2_df.index + 1

# Reorder the DataFrame to have 'Sample' and 'Agent' as the first two columns
column_order = ['Sample', 'Agent'] + [col for col in sample2_df.columns if col not in ['Sample', 'Agent']]
sample2_df = sample2_df[column_order]

# Display the first few rows of the modified DataFrame to verify
print(sample2_df.head())

In [ ]:
# Export the DataFrame to a CSV file
sample2_df.to_csv('sample2.csv', index=False)

<br>

---

<br>

### Sample 3: Creating Dataframe for Agent Traits and Survey Responses

In [ ]:
result3_1_df = result3_1.to_pandas()[['agent.personas', 'agent.doomers', 'agent.age', 'agent.gender', 'agent.education',
	'agent.occupation', 'agent.income', 'agent.nationality_politics', 'agent.politics', 'agent.economics',
	'answer.q1_cb']]

In [ ]:
result3_2_df = result3_2.to_pandas()[['answer.q2_cb']]
result3_3_df = result3_3.to_pandas()[['answer.q3_mc']]
result3_4_df = result3_4.to_pandas()[['answer.q4_ls']]

result3_10_df = result3_10.to_pandas()[['answer.q10_mc']]
result3_11_df = result3_11.to_pandas()[['answer.q11_mc']]
result3_12_df = result3_12.to_pandas()[['answer.q12_mc']]
result3_13_df = result3_13.to_pandas()[['answer.q13_mc']]
result3_14_df = result3_14.to_pandas()[['answer.q14_mc']]
result3_21_df = result3_21.to_pandas()[['answer.q21_mc']]

result3_5_pre_df = result3_5_pre.to_pandas()[['answer.pre5']]
result3_6_pre_df = result3_6_pre.to_pandas()[['answer.pre6']]
result3_7_pre_df = result3_7_pre.to_pandas()[['answer.pre7']]
result3_8_pre_df = result3_8_pre.to_pandas()[['answer.pre8']]
result3_9_pre_df = result3_9_pre.to_pandas()[['answer.pre9']]

result3_17_pre_df = result3_17_pre.to_pandas()[['answer.pre17']]
result3_18_pre_df = result3_18_pre.to_pandas()[['answer.pre18']]
result3_19_pre_df = result3_19_pre.to_pandas()[['answer.pre19']]
result3_20_pre_df = result3_20_pre.to_pandas()[['answer.pre20']]
result3_22_pre_df = result3_22_pre.to_pandas()[['answer.pre22']]

result3_5_post_df = result3_5_post.to_pandas()[['answer.post5']]
result3_6_post_df = result3_6_post.to_pandas()[['answer.post6']]
result3_7_post_df = result3_7_post.to_pandas()[['answer.post7']]
result3_8_post_df = result3_8_post.to_pandas()[['answer.post8']]
result3_9_post_df = result3_9_post.to_pandas()[['answer.post9']]

result3_17_post_df = result3_17_post.to_pandas()[['answer.post17']]
result3_18_post_df = result3_18_post.to_pandas()[['answer.post18']]
result3_19_post_df = result3_19_post.to_pandas()[['answer.post19']]
result3_20_post_df = result3_20_post.to_pandas()[['answer.post20']]
result3_22_post_df = result3_22_post.to_pandas()[['answer.post22']]

In [ ]:
# Concatenate dataframess column-wise
sample3_df = pd.concat([result3_1_df, result3_2_df, result3_3_df, result3_4_df, result3_10_df, result3_11_df, result3_12_df, result3_13_df, result3_14_df, result3_21_df, result3_5_pre_df, result3_6_pre_df, result3_7_pre_df, result3_8_pre_df, result3_9_pre_df, result3_17_pre_df, result3_18_pre_df, result3_19_pre_df, result3_20_pre_df, result3_22_pre_df, result3_5_post_df, result3_6_post_df, result3_7_post_df, result3_8_post_df, result3_9_post_df, result3_17_post_df, result3_18_post_df, result3_19_post_df, result3_20_post_df, result3_22_post_df], axis=1)

In [ ]:
# Rename the columns
sample3_df = sample3_df.rename(columns={"personas": "Persona", "doomers": "Doomer Exposure", "age": "Age", "gender":"Gender", "education": "Educational Attainment", "income":"Annual Pre-Tax Income", "occupation": "Occupation", "nationality_politics": "Nationality and US Political Party Preference", "politics": "Political Orientation", "economics": "Economic Views"})

# Add the 'Sample' column with all values set to 3
sample3_df['Sample'] = 3

# Add the 'Agent' column, which is the index + 1
sample3_df['Agent'] = sample3_df.index + 1

# Reorder the DataFrame to have 'Sample' and 'Agent' as the first two columns
column_order = ['Sample', 'Agent'] + [col for col in sample3_df.columns if col not in ['Sample', 'Agent']]
sample3_df = sample3_df[column_order]

# Display the first few rows of the modified DataFrame to verify
print(sample3_df.head())

In [252]:
# Export the DataFrame to a CSV file
sample3_df.to_csv('sample3.csv', index=False)

<br>

---

<br>

### Sample 4: Creating Dataframe for Agent Traits and Survey Responses

In [ ]:
result4_1_df = result4_1.to_pandas()[['agent.personas', 'agent.doomers', 'agent.age', 'agent.gender', 'agent.education',
	'agent.occupation', 'agent.income', 'agent.nationality_politics', 'agent.politics', 'agent.economics',
	'answer.q1_cb']]

In [ ]:
result4_2_df = result4_2.to_pandas()[['answer.q2_cb']]
result4_3_df = result4_3.to_pandas()[['answer.q3_mc']]
result4_4_df = result4_4.to_pandas()[['answer.q4_ls']]

result4_10_df = result4_10.to_pandas()[['answer.q10_mc']]
result4_11_df = result4_11.to_pandas()[['answer.q11_mc']]
result4_12_df = result4_12.to_pandas()[['answer.q12_mc']]
result4_13_df = result4_13.to_pandas()[['answer.q13_mc']]
result4_14_df = result4_14.to_pandas()[['answer.q14_mc']]
result4_21_df = result4_21.to_pandas()[['answer.q21_mc']]

result4_5_pre_df = result4_5_pre.to_pandas()[['answer.pre5']]
result4_6_pre_df = result4_6_pre.to_pandas()[['answer.pre6']]
result4_7_pre_df = result4_7_pre.to_pandas()[['answer.pre7']]
result4_8_pre_df = result4_8_pre.to_pandas()[['answer.pre8']]
result4_9_pre_df = result4_9_pre.to_pandas()[['answer.pre9']]

result4_17_pre_df = result4_17_pre.to_pandas()[['answer.pre17']]
result4_18_pre_df = result4_18_pre.to_pandas()[['answer.pre18']]
result4_19_pre_df = result4_19_pre.to_pandas()[['answer.pre19']]
result4_20_pre_df = result4_20_pre.to_pandas()[['answer.pre20']]
result4_22_pre_df = result4_22_pre.to_pandas()[['answer.pre22']]

result4_5_post_df = result4_5_post.to_pandas()[['answer.post5']]
result4_6_post_df = result4_6_post.to_pandas()[['answer.post6']]
result4_7_post_df = result4_7_post.to_pandas()[['answer.post7']]
result4_8_post_df = result4_8_post.to_pandas()[['answer.post8']]
result4_9_post_df = result4_9_post.to_pandas()[['answer.post9']]

result4_17_post_df = result4_17_post.to_pandas()[['answer.post17']]
result4_18_post_df = result4_18_post.to_pandas()[['answer.post18']]
result4_19_post_df = result4_19_post.to_pandas()[['answer.post19']]
result4_20_post_df = result4_20_post.to_pandas()[['answer.post20']]
result4_22_post_df = result4_22_post.to_pandas()[['answer.post22']]

In [ ]:
# Concatenate dataframess column-wise
sample4_df = pd.concat([result4_1_df, result4_2_df, result4_3_df, result4_4_df, result4_10_df, result4_11_df, result4_12_df, result4_13_df, result4_14_df, result4_21_df, result4_5_pre_df, result4_6_pre_df, result4_7_pre_df, result4_8_pre_df, result4_9_pre_df, result4_17_pre_df, result4_18_pre_df, result4_19_pre_df, result4_20_pre_df, result4_22_pre_df, result4_5_post_df, result4_6_post_df, result4_7_post_df, result4_8_post_df, result4_9_post_df, result4_17_post_df, result4_18_post_df, result4_19_post_df, result4_20_post_df, result4_22_post_df], axis=1)

In [ ]:
# Rename the columns
sample4_df = sample4_df.rename(columns={"personas": "Persona", "doomers": "Doomer Exposure", "age": "Age", "gender":"Gender", "education": "Educational Attainment", "income":"Annual Pre-Tax Income", "occupation": "Occupation", "nationality_politics": "Nationality and US Political Party Preference", "politics": "Political Orientation", "economics": "Economic Views"})

# Add the 'Sample' column with all values set to 4
sample4_df['Sample'] = 4

# Add the 'Agent' column, which is the index + 1
sample4_df['Agent'] = sample4_df.index + 1

# Reorder the DataFrame to have 'Sample' and 'Agent' as the first two columns
column_order = ['Sample', 'Agent'] + [col for col in sample4_df.columns if col not in ['Sample', 'Agent']]
sample4_df = sample4_df[column_order]

# Display the first few rows of the modified DataFrame to verify
print(sample4_df.head())

In [253]:
# Export the DataFrame to a CSV file
sample4_df.to_csv('sample4.csv', index=False)

<br>

---

<br>

### Sample 5: Creating Dataframe for Agent Traits and Survey Responses

In [214]:
result5_1_df = result5_1.to_pandas()[['agent.personas', 'agent.doomers', 'agent.age', 'agent.gender', 'agent.education',
	'agent.occupation', 'agent.income', 'agent.nationality_politics', 'agent.politics', 'agent.economics',
	'answer.q1_cb']]

In [ ]:
result5_2_df = result5_2.to_pandas()[['answer.q2_cb']]
result5_3_df = result5_3.to_pandas()[['answer.q3_mc']]
result5_4_df = result5_4.to_pandas()[['answer.q4_ls']]

result5_10_df = result5_10.to_pandas()[['answer.q10_mc']]
result5_11_df = result5_11.to_pandas()[['answer.q11_mc']]
result5_12_df = result5_12.to_pandas()[['answer.q12_mc']]
result5_13_df = result5_13.to_pandas()[['answer.q13_mc']]
result5_14_df = result5_14.to_pandas()[['answer.q14_mc']]
result5_21_df = result5_21.to_pandas()[['answer.q21_mc']]

result5_5_pre_df = result5_5_pre.to_pandas()[['answer.pre5']]
result5_6_pre_df = result5_6_pre.to_pandas()[['answer.pre6']]
result5_7_pre_df = result5_7_pre.to_pandas()[['answer.pre7']]
result5_8_pre_df = result5_8_pre.to_pandas()[['answer.pre8']]
result5_9_pre_df = result5_9_pre.to_pandas()[['answer.pre9']]

result5_17_pre_df = result5_17_pre.to_pandas()[['answer.pre17']]
result5_18_pre_df = result5_18_pre.to_pandas()[['answer.pre18']]
result5_19_pre_df = result5_19_pre.to_pandas()[['answer.pre19']]
result5_20_pre_df = result5_20_pre.to_pandas()[['answer.pre20']]
result5_22_pre_df = result5_22_pre.to_pandas()[['answer.pre22']]

result5_5_post_df = result5_5_post.to_pandas()[['answer.post5']]
result5_6_post_df = result5_6_post.to_pandas()[['answer.post6']]
result5_7_post_df = result5_7_post.to_pandas()[['answer.post7']]
result5_8_post_df = result5_8_post.to_pandas()[['answer.post8']]
result5_9_post_df = result5_9_post.to_pandas()[['answer.post9']]

result5_17_post_df = result5_17_post.to_pandas()[['answer.post17']]
result5_18_post_df = result5_18_post.to_pandas()[['answer.post18']]
result5_19_post_df = result5_19_post.to_pandas()[['answer.post19']]
result5_20_post_df = result5_20_post.to_pandas()[['answer.post20']]
result5_22_post_df = result5_22_post.to_pandas()[['answer.post22']]

In [ ]:
# Concatenate dataframess column-wise
sample5_df = pd.concat([result5_1_df, result5_2_df, result5_3_df, result5_4_df, result5_10_df, result5_11_df, result5_12_df, result5_13_df, result5_14_df, result5_21_df, result5_5_pre_df, result5_6_pre_df, result5_7_pre_df, result5_8_pre_df, result5_9_pre_df, result5_17_pre_df, result5_18_pre_df, result5_19_pre_df, result5_20_pre_df, result5_22_pre_df, result5_5_post_df, result5_6_post_df, result5_7_post_df, result5_8_post_df, result5_9_post_df, result5_17_post_df, result5_18_post_df, result5_19_post_df, result5_20_post_df, result5_22_post_df], axis=1)

In [ ]:
# Rename the columns
sample5_df = sample5_df.rename(columns={"personas": "Persona", "doomers": "Doomer Exposure", "age": "Age", "gender":"Gender", "education": "Educational Attainment", "income":"Annual Pre-Tax Income", "occupation": "Occupation", "nationality_politics": "Nationality and US Political Party Preference", "politics": "Political Orientation", "economics": "Economic Views"})

# Add the 'Sample' column with all values set to 5
sample5_df['Sample'] = 5

# Add the 'Agent' column, which is the index + 1
sample5_df['Agent'] = sample5_df.index + 1

# Reorder the DataFrame to have 'Sample' and 'Agent' as the first two columns
column_order = ['Sample', 'Agent'] + [col for col in sample5_df.columns if col not in ['Sample', 'Agent']]
sample5_df = sample5_df[column_order]

# Display the first few rows of the modified DataFrame to verify
print(sample5_df.head())

In [254]:
# Export the DataFrame to a CSV file
sample5_df.to_csv('sample5.csv', index=False)

<br>

---

<br>

### Sample 6: Creating Dataframe for Agent Traits and Survey Responses

In [ ]:
result6_1_df = result6_1.to_pandas()[['agent.personas', 'agent.doomers', 'agent.age', 'agent.gender', 'agent.education',
	'agent.occupation', 'agent.income', 'agent.nationality_politics', 'agent.politics', 'agent.economics',
	'answer.q1_cb']]

In [ ]:
result6_2_df = result6_2.to_pandas()[['answer.q2_cb']]
result6_3_df = result6_3.to_pandas()[['answer.q3_mc']]
result6_4_df = result6_4.to_pandas()[['answer.q4_ls']]

result6_10_df = result6_10.to_pandas()[['answer.q10_mc']]
result6_11_df = result6_11.to_pandas()[['answer.q11_mc']]
result6_12_df = result6_12.to_pandas()[['answer.q12_mc']]
result6_13_df = result6_13.to_pandas()[['answer.q13_mc']]
result6_14_df = result6_14.to_pandas()[['answer.q14_mc']]
result6_21_df = result6_21.to_pandas()[['answer.q21_mc']]

result6_5_pre_df = result6_5_pre.to_pandas()[['answer.pre5']]
result6_6_pre_df = result6_6_pre.to_pandas()[['answer.pre6']]
result6_7_pre_df = result6_7_pre.to_pandas()[['answer.pre7']]
result6_8_pre_df = result6_8_pre.to_pandas()[['answer.pre8']]
result6_9_pre_df = result6_9_pre.to_pandas()[['answer.pre9']]

result6_17_pre_df = result6_17_pre.to_pandas()[['answer.pre17']]
result6_18_pre_df = result6_18_pre.to_pandas()[['answer.pre18']]
result6_19_pre_df = result6_19_pre.to_pandas()[['answer.pre19']]
result6_20_pre_df = result6_20_pre.to_pandas()[['answer.pre20']]
result6_22_pre_df = result6_22_pre.to_pandas()[['answer.pre22']]

result6_5_post_df = result6_5_post.to_pandas()[['answer.post5']]
result6_6_post_df = result6_6_post.to_pandas()[['answer.post6']]
result6_7_post_df = result6_7_post.to_pandas()[['answer.post7']]
result6_8_post_df = result6_8_post.to_pandas()[['answer.post8']]
result6_9_post_df = result6_9_post.to_pandas()[['answer.post9']]

result6_17_post_df = result6_17_post.to_pandas()[['answer.post17']]
result6_18_post_df = result6_18_post.to_pandas()[['answer.post18']]
result6_19_post_df = result6_19_post.to_pandas()[['answer.post19']]
result6_20_post_df = result6_20_post.to_pandas()[['answer.post20']]
result6_22_post_df = result6_22_post.to_pandas()[['answer.post22']]

In [215]:
# Concatenate dataframess column-wise
sample6_df = pd.concat([result6_1_df, result6_2_df, result6_3_df, result6_4_df, result6_10_df, result6_11_df, result6_12_df, result6_13_df, result6_14_df, result6_21_df, result6_5_pre_df, result6_6_pre_df, result6_7_pre_df, result6_8_pre_df, result6_9_pre_df, result6_17_pre_df, result6_18_pre_df, result6_19_pre_df, result6_20_pre_df, result6_22_pre_df, result6_5_post_df, result6_6_post_df, result6_7_post_df, result6_8_post_df, result6_9_post_df, result6_17_post_df, result6_18_post_df, result6_19_post_df, result6_20_post_df, result6_22_post_df], axis=1)

In [ ]:
# Rename the columns
sample6_df = sample6_df.rename(columns={"personas": "Persona", "doomers": "Doomer Exposure", "age": "Age", "gender":"Gender", "education": "Educational Attainment", "income":"Annual Pre-Tax Income", "occupation": "Occupation", "nationality_politics": "Nationality and US Political Party Preference", "politics": "Political Orientation", "economics": "Economic Views"})

# Add the 'Sample' column with all values set to 6
sample6_df['Sample'] = 6

# Add the 'Agent' column, which is the index + 1
sample6_df['Agent'] = sample6_df.index + 1

# Reorder the DataFrame to have 'Sample' and 'Agent' as the first two columns
column_order = ['Sample', 'Agent'] + [col for col in sample6_df.columns if col not in ['Sample', 'Agent']]
sample6_df = sample6_df[column_order]

# Display the first few rows of the modified DataFrame to verify
print(sample6_df.head())

In [255]:
# Export the DataFrame to a CSV file
sample6_df.to_csv('sample6.csv', index=False)

<br>

---

<br>

### Sample 7: Creating Dataframe for Agent Traits and Survey Responses

In [216]:
result7_1_df = result7_1.to_pandas()[['agent.personas', 'agent.doomers', 'agent.age', 'agent.gender', 'agent.education',
	'agent.occupation', 'agent.income', 'agent.nationality_politics', 'agent.politics', 'agent.economics',
	'answer.q1_cb']]

In [ ]:
result7_2_df = result7_2.to_pandas()[['answer.q2_cb']]
result7_3_df = result7_3.to_pandas()[['answer.q3_mc']]
result7_4_df = result7_4.to_pandas()[['answer.q4_ls']]

result7_10_df = result7_10.to_pandas()[['answer.q10_mc']]
result7_11_df = result7_11.to_pandas()[['answer.q11_mc']]
result7_12_df = result7_12.to_pandas()[['answer.q12_mc']]
result7_13_df = result7_13.to_pandas()[['answer.q13_mc']]
result7_14_df = result7_14.to_pandas()[['answer.q14_mc']]
result7_21_df = result7_21.to_pandas()[['answer.q21_mc']]

result7_5_pre_df = result7_5_pre.to_pandas()[['answer.pre5']]
result7_6_pre_df = result7_6_pre.to_pandas()[['answer.pre6']]
result7_7_pre_df = result7_7_pre.to_pandas()[['answer.pre7']]
result7_8_pre_df = result7_8_pre.to_pandas()[['answer.pre8']]
result7_9_pre_df = result7_9_pre.to_pandas()[['answer.pre9']]

result7_17_pre_df = result7_17_pre.to_pandas()[['answer.pre17']]
result7_18_pre_df = result7_18_pre.to_pandas()[['answer.pre18']]
result7_19_pre_df = result7_19_pre.to_pandas()[['answer.pre19']]
result7_20_pre_df = result7_20_pre.to_pandas()[['answer.pre20']]
result7_22_pre_df = result7_22_pre.to_pandas()[['answer.pre22']]

result7_5_post_df = result7_5_post.to_pandas()[['answer.post5']]
result7_6_post_df = result7_6_post.to_pandas()[['answer.post6']]
result7_7_post_df = result7_7_post.to_pandas()[['answer.post7']]
result7_8_post_df = result7_8_post.to_pandas()[['answer.post8']]
result7_9_post_df = result7_9_post.to_pandas()[['answer.post9']]

result7_17_post_df = result7_17_post.to_pandas()[['answer.post17']]
result7_18_post_df = result7_18_post.to_pandas()[['answer.post18']]
result7_19_post_df = result7_19_post.to_pandas()[['answer.post19']]
result7_20_post_df = result7_20_post.to_pandas()[['answer.post20']]
result7_22_post_df = result7_22_post.to_pandas()[['answer.post22']]

In [ ]:
# Concatenate dataframess column-wise
sample7_df = pd.concat([result7_1_df, result7_2_df, result7_3_df, result7_4_df, result7_10_df, result7_11_df, result7_12_df, result7_13_df, result7_14_df, result7_21_df, result7_5_pre_df, result7_6_pre_df, result7_7_pre_df, result7_8_pre_df, result7_9_pre_df, result7_17_pre_df, result7_18_pre_df, result7_19_pre_df, result7_20_pre_df, result7_22_pre_df, result7_5_post_df, result7_6_post_df, result7_7_post_df, result7_8_post_df, result7_9_post_df, result7_17_post_df, result7_18_post_df, result7_19_post_df, result7_20_post_df, result7_22_post_df], axis=1)

In [ ]:
# Rename the columns
sample7_df = sample7_df.rename(columns={"personas": "Persona", "doomers": "Doomer Exposure", "age": "Age", "gender":"Gender", "education": "Educational Attainment", "income":"Annual Pre-Tax Income", "occupation": "Occupation", "nationality_politics": "Nationality and US Political Party Preference", "politics": "Political Orientation", "economics": "Economic Views"})

# Add the 'Sample' column with all values set to 7
sample7_df['Sample'] = 7

# Add the 'Agent' column, which is the index + 1
sample7_df['Agent'] = sample7_df.index + 1

# Reorder the DataFrame to have 'Sample' and 'Agent' as the first two columns
column_order = ['Sample', 'Agent'] + [col for col in sample7_df.columns if col not in ['Sample', 'Agent']]
sample7_df = sample7_df[column_order]

# Display the first few rows of the modified DataFrame to verify
print(sample7_df.head())

In [256]:
# Export the DataFrame to a CSV file
sample7_df.to_csv('sample7.csv', index=False)

<br>

---

<br>

### Sample 8: Creating Dataframe for Agent Traits and Survey Responses

In [217]:
result8_1_df = result8_1.to_pandas()[['agent.personas', 'agent.doomers', 'agent.age', 'agent.gender', 'agent.education',
	'agent.occupation', 'agent.income', 'agent.nationality_politics', 'agent.politics', 'agent.economics',
	'answer.q1_cb']]

In [ ]:
result8_2_df = result8_2.to_pandas()[['answer.q2_cb']]
result8_3_df = result8_3.to_pandas()[['answer.q3_mc']]
result8_4_df = result8_4.to_pandas()[['answer.q4_ls']]

result8_10_df = result8_10.to_pandas()[['answer.q10_mc']]
result8_11_df = result8_11.to_pandas()[['answer.q11_mc']]
result8_12_df = result8_12.to_pandas()[['answer.q12_mc']]
result8_13_df = result8_13.to_pandas()[['answer.q13_mc']]
result8_14_df = result8_14.to_pandas()[['answer.q14_mc']]
result8_21_df = result8_21.to_pandas()[['answer.q21_mc']]

result8_5_pre_df = result8_5_pre.to_pandas()[['answer.pre5']]
result8_6_pre_df = result8_6_pre.to_pandas()[['answer.pre6']]
result8_7_pre_df = result8_7_pre.to_pandas()[['answer.pre7']]
result8_8_pre_df = result8_8_pre.to_pandas()[['answer.pre8']]
result8_9_pre_df = result8_9_pre.to_pandas()[['answer.pre9']]

result8_17_pre_df = result8_17_pre.to_pandas()[['answer.pre17']]
result8_18_pre_df = result8_18_pre.to_pandas()[['answer.pre18']]
result8_19_pre_df = result8_19_pre.to_pandas()[['answer.pre19']]
result8_20_pre_df = result8_20_pre.to_pandas()[['answer.pre20']]
result8_22_pre_df = result8_22_pre.to_pandas()[['answer.pre22']]

result8_5_post_df = result8_5_post.to_pandas()[['answer.post5']]
result8_6_post_df = result8_6_post.to_pandas()[['answer.post6']]
result8_7_post_df = result8_7_post.to_pandas()[['answer.post7']]
result8_8_post_df = result8_8_post.to_pandas()[['answer.post8']]
result8_9_post_df = result8_9_post.to_pandas()[['answer.post9']]

result8_17_post_df = result8_17_post.to_pandas()[['answer.post17']]
result8_18_post_df = result8_18_post.to_pandas()[['answer.post18']]
result8_19_post_df = result8_19_post.to_pandas()[['answer.post19']]
result8_20_post_df = result8_20_post.to_pandas()[['answer.post20']]
result8_22_post_df = result8_22_post.to_pandas()[['answer.post22']]

In [ ]:
# Concatenate dataframess column-wise
sample8_df = pd.concat([result8_1_df, result8_2_df, result8_3_df, result8_4_df, result8_10_df, result8_11_df, result8_12_df, result8_13_df, result8_14_df, result8_21_df, result8_5_pre_df, result8_6_pre_df, result8_7_pre_df, result8_8_pre_df, result8_9_pre_df, result8_17_pre_df, result8_18_pre_df, result8_19_pre_df, result8_20_pre_df, result8_22_pre_df, result8_5_post_df, result8_6_post_df, result8_7_post_df, result8_8_post_df, result8_9_post_df, result8_17_post_df, result8_18_post_df, result8_19_post_df, result8_20_post_df, result8_22_post_df], axis=1)

In [ ]:
# Rename the columns
sample8_df = sample8_df.rename(columns={"personas": "Persona", "doomers": "Doomer Exposure", "age": "Age", "gender":"Gender", "education": "Educational Attainment", "income":"Annual Pre-Tax Income", "occupation": "Occupation", "nationality_politics": "Nationality and US Political Party Preference", "politics": "Political Orientation", "economics": "Economic Views"})

# Add the 'Sample' column with all values set to 8
sample8_df['Sample'] = 8

# Add the 'Agent' column, which is the index + 1
sample8_df['Agent'] = sample8_df.index + 1

# Reorder the DataFrame to have 'Sample' and 'Agent' as the first two columns
column_order = ['Sample', 'Agent'] + [col for col in sample8_df.columns if col not in ['Sample', 'Agent']]
sample8_df = sample8_df[column_order]

# Display the first few rows of the modified DataFrame to verify
print(sample8_df.head())

In [257]:
# Export the DataFrame to a CSV file
sample8_df.to_csv('sample8.csv', index=False)

<br>

---

<br>

### Sample 9: Creating Dataframe for Agent Traits and Survey Responses

In [ ]:
result9_1_df = result9_1.to_pandas()[['agent.personas', 'agent.doomers', 'agent.age', 'agent.gender', 'agent.education',
	'agent.occupation', 'agent.income', 'agent.nationality_politics', 'agent.politics', 'agent.economics',
	'answer.q1_cb']]

In [ ]:
result9_2_df = result9_2.to_pandas()[['answer.q2_cb']]
result9_3_df = result9_3.to_pandas()[['answer.q3_mc']]
result9_4_df = result9_4.to_pandas()[['answer.q4_ls']]

result9_10_df = result9_10.to_pandas()[['answer.q10_mc']]
result9_11_df = result9_11.to_pandas()[['answer.q11_mc']]
result9_12_df = result9_12.to_pandas()[['answer.q12_mc']]
result9_13_df = result9_13.to_pandas()[['answer.q13_mc']]
result9_14_df = result9_14.to_pandas()[['answer.q14_mc']]
result9_21_df = result9_21.to_pandas()[['answer.q21_mc']]

result9_5_pre_df = result9_5_pre.to_pandas()[['answer.pre5']]
result9_6_pre_df = result9_6_pre.to_pandas()[['answer.pre6']]
result9_7_pre_df = result9_7_pre.to_pandas()[['answer.pre7']]
result9_8_pre_df = result9_8_pre.to_pandas()[['answer.pre8']]
result9_9_pre_df = result9_9_pre.to_pandas()[['answer.pre9']]

result9_17_pre_df = result9_17_pre.to_pandas()[['answer.pre17']]
result9_18_pre_df = result9_18_pre.to_pandas()[['answer.pre18']]
result9_19_pre_df = result9_19_pre.to_pandas()[['answer.pre19']]
result9_20_pre_df = result9_20_pre.to_pandas()[['answer.pre20']]
result9_22_pre_df = result9_22_pre.to_pandas()[['answer.pre22']]

result9_5_post_df = result9_5_post.to_pandas()[['answer.post5']]
result9_6_post_df = result9_6_post.to_pandas()[['answer.post6']]
result9_7_post_df = result9_7_post.to_pandas()[['answer.post7']]
result9_8_post_df = result9_8_post.to_pandas()[['answer.post8']]
result9_9_post_df = result9_9_post.to_pandas()[['answer.post9']]

result9_17_post_df = result9_17_post.to_pandas()[['answer.post17']]
result9_18_post_df = result9_18_post.to_pandas()[['answer.post18']]
result9_19_post_df = result9_19_post.to_pandas()[['answer.post19']]
result9_20_post_df = result9_20_post.to_pandas()[['answer.post20']]
result9_22_post_df = result9_22_post.to_pandas()[['answer.post22']]

In [218]:
# Concatenate dataframess column-wise
sample9_df = pd.concat([result9_1_df, result9_2_df, result9_3_df, result9_4_df, result9_10_df, result9_11_df, result9_12_df, result9_13_df, result9_14_df, result9_21_df, result9_5_pre_df, result9_6_pre_df, result9_7_pre_df, result9_8_pre_df, result9_9_pre_df, result9_17_pre_df, result9_18_pre_df, result9_19_pre_df, result9_20_pre_df, result9_22_pre_df, result9_5_post_df, result9_6_post_df, result9_7_post_df, result9_8_post_df, result9_9_post_df, result9_17_post_df, result9_18_post_df, result9_19_post_df, result9_20_post_df, result9_22_post_df], axis=1)

In [ ]:
# Rename the columns
sample9_df = sample9_df.rename(columns={"personas": "Persona", "doomers": "Doomer Exposure", "age": "Age", "gender":"Gender", "education": "Educational Attainment", "income":"Annual Pre-Tax Income", "occupation": "Occupation", "nationality_politics": "Nationality and US Political Party Preference", "politics": "Political Orientation", "economics": "Economic Views"})

# Add the 'Sample' column with all values set to 9
sample9_df['Sample'] = 9

# Add the 'Agent' column, which is the index + 1
sample9_df['Agent'] = sample9_df.index + 1

# Reorder the DataFrame to have 'Sample' and 'Agent' as the first two columns
column_order = ['Sample', 'Agent'] + [col for col in sample9_df.columns if col not in ['Sample', 'Agent']]
sample9_df = sample9_df[column_order]

# Display the first few rows of the modified DataFrame to verify
print(sample9_df.head())

In [258]:
# Export the DataFrame to a CSV file
sample9_df.to_csv('sample9.csv', index=False)

<br>

---

<br>

### Sample 10: Creating Dataframe for Agent Traits and Survey Responses

In [219]:
result10_1_df = result10_1.to_pandas()[['agent.personas', 'agent.doomers', 'agent.age', 'agent.gender', 'agent.education',
	'agent.occupation', 'agent.income', 'agent.nationality_politics', 'agent.politics', 'agent.economics',
	'answer.q1_cb']]

In [ ]:
result10_2_df = result10_2.to_pandas()[['answer.q2_cb']]
result10_3_df = result10_3.to_pandas()[['answer.q3_mc']]
result10_4_df = result10_4.to_pandas()[['answer.q4_ls']]

result10_10_df = result10_10.to_pandas()[['answer.q10_mc']]
result10_11_df = result10_11.to_pandas()[['answer.q11_mc']]
result10_12_df = result10_12.to_pandas()[['answer.q12_mc']]
result10_13_df = result10_13.to_pandas()[['answer.q13_mc']]
result10_14_df = result10_14.to_pandas()[['answer.q14_mc']]
result10_21_df = result10_21.to_pandas()[['answer.q21_mc']]

result10_5_pre_df = result10_5_pre.to_pandas()[['answer.pre5']]
result10_6_pre_df = result10_6_pre.to_pandas()[['answer.pre6']]
result10_7_pre_df = result10_7_pre.to_pandas()[['answer.pre7']]
result10_8_pre_df = result10_8_pre.to_pandas()[['answer.pre8']]
result10_9_pre_df = result10_9_pre.to_pandas()[['answer.pre9']]

result10_17_pre_df = result10_17_pre.to_pandas()[['answer.pre17']]
result10_18_pre_df = result10_18_pre.to_pandas()[['answer.pre18']]
result10_19_pre_df = result10_19_pre.to_pandas()[['answer.pre19']]
result10_20_pre_df = result10_20_pre.to_pandas()[['answer.pre20']]
result10_22_pre_df = result10_22_pre.to_pandas()[['answer.pre22']]

result10_5_post_df = result10_5_post.to_pandas()[['answer.post5']]
result10_6_post_df = result10_6_post.to_pandas()[['answer.post6']]
result10_7_post_df = result10_7_post.to_pandas()[['answer.post7']]
result10_8_post_df = result10_8_post.to_pandas()[['answer.post8']]
result10_9_post_df = result10_9_post.to_pandas()[['answer.post9']]

result10_17_post_df = result10_17_post.to_pandas()[['answer.post17']]
result10_18_post_df = result10_18_post.to_pandas()[['answer.post18']]
result10_19_post_df = result10_19_post.to_pandas()[['answer.post19']]
result10_20_post_df = result10_20_post.to_pandas()[['answer.post20']]
result10_22_post_df = result10_22_post.to_pandas()[['answer.post22']]

In [ ]:
# Concatenate dataframess column-wise
sample10_df = pd.concat([result10_1_df, result10_2_df, result10_3_df, result10_4_df, result10_10_df, result10_11_df, result10_12_df, result10_13_df, result10_14_df, result10_21_df, result10_5_pre_df, result10_6_pre_df, result10_7_pre_df, result10_8_pre_df, result10_9_pre_df, result10_17_pre_df, result10_18_pre_df, result10_19_pre_df, result10_20_pre_df, result10_22_pre_df, result10_5_post_df, result10_6_post_df, result10_7_post_df, result10_8_post_df, result10_9_post_df, result10_17_post_df, result10_18_post_df, result10_19_post_df, result10_20_post_df, result10_22_post_df], axis=1)

In [ ]:
# Rename the columns
sample10_df = sample10_df.rename(columns={"personas": "Persona", "doomers": "Doomer Exposure", "age": "Age", "gender":"Gender", "education": "Educational Attainment", "income":"Annual Pre-Tax Income", "occupation": "Occupation", "nationality_politics": "Nationality and US Political Party Preference", "politics": "Political Orientation", "economics": "Economic Views"})

# Add the 'Sample' column with all values set to 10
sample10_df['Sample'] = 10

# Add the 'Agent' column, which is the index + 1
sample10_df['Agent'] = sample10_df.index + 1

# Reorder the DataFrame to have 'Sample' and 'Agent' as the first two columns
column_order = ['Sample', 'Agent'] + [col for col in sample10_df.columns if col not in ['Sample', 'Agent']]
sample10_df = sample10_df[column_order]

# Display the first few rows of the modified DataFrame to verify
print(sample10_df.head())

In [259]:
# Export the DataFrame to a CSV file
sample10_df.to_csv('sample10.csv', index=False)

<br>

---

<br>

## Data Analysis

## Results

**General HTML Table with Headers**: <br>

| Header 1 | Header 2 |
|------|------|
| row 1, cell 1 | row 1, cell 2 |
| row 2, cell 1 | row 2, cell 2 |